In [4613]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones
import re

from unidecode import unidecode
import unicodedata
import datetime
import os
import gc


# visuzalizaciones
import missingno as msno
import matplotlib as plt
import plotly.express as px

In [4614]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows',80)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

# Funciones

In [4615]:


# Función para normalizar texto
def normalize_text(text):
    # Reemplazar caracteres especiales y convertir a minúsculas para una comparación consistente
    text = text.replace("/", " ")  # Reemplazar la barra por espacio o eliminarla completamente
    text = text.replace("ñ", "n")  # Reemplazar la "ñ" por "n"
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    ).lower()

In [4616]:
def normalizar(texto):
 return unidecode(texto).lower()

In [4617]:
def actualizar_valores(df, df1, clave, columna_a_actualizar):
    """
    Actualiza los valores de una columna en un dataframe utilizando los valores de otro dataframe.

    :param df: DataFrame a actualizar.
    :param df1: DataFrame que contiene los valores correctos.
    :param clave: Nombre de la columna que funciona como clave primaria para hacer la correspondencia.
    :param columna_a_actualizar: Nombre de la columna que será actualizada en df.
    :return: DataFrame actualizado.
    """
    # Crear un diccionario con los valores correctos del segundo dataframe
    valores_correctos = df1.set_index(clave)[columna_a_actualizar].to_dict()

    # Actualizar los valores en el dataframe principal
    df[columna_a_actualizar] = df[clave].map(valores_correctos).fillna(df[columna_a_actualizar])

    return df


In [4618]:
def actualizar_valores_conclave(df, df1, clave, columna_a_actualizar):
    """
    Actualiza los valores de una columna en un dataframe utilizando los valores de otro dataframe.

    :param df: DataFrame a actualizar.
    :param df1: DataFrame que contiene los valores correctos.
    :param clave: Nombre de la columna que funciona como clave primaria para hacer la correspondencia.
    :param columna_a_actualizar: Nombre de la columna que será actualizada en df.
    :return: DataFrame actualizado.
    """
    # Crear un diccionario con los valores correctos del segundo dataframe
    valores_correctos = df1.set_index(clave)[columna_a_actualizar].to_dict()

    # Actualizar los valores en el dataframe principal solo si coinciden las claves
    df[columna_a_actualizar] = df[clave].map(valores_correctos).combine_first(df[columna_a_actualizar])

    return df


In [4619]:
def obtener_valores_por_columna(df, columna, valor):
    """
    Devuelve los valores de las demás columnas de la fila donde la columna especificada tiene el valor dado.

    :param df: DataFrame de pandas
    :param columna: Nombre de la columna para buscar el valor
    :param valor: Valor único para buscar en la columna
    :return: Diccionario con los valores de las demás columnas
    """
    # Convertir los valores de la columna a cadenas de texto y eliminar espacios en blanco
    df[columna] = df[columna].astype(str).str.strip()
    valor = str(valor).strip()

    fila = df.loc[df[columna] == valor]
    if not fila.empty:
        return fila.iloc[0].to_dict()
    else:
        return {}


In [4620]:
def porcentaje_nulos(df, columnas):
    if isinstance(columnas, str):
        columnas = [columnas]  # Convertir a lista si es una sola columna
    resultados = {}
    for columna in columnas:
        if columna in df.columns:
            total_filas = len(df)
            nulos = df[columna].isnull().sum()
            porcentaje_nulos = (nulos / total_filas) * 100
            resultados[columna] = porcentaje_nulos
        else:
            resultados[columna] = 'Column not found'
    return resultados



In [4621]:

# Función para clasificar el tipo de calefacción y asignar el nombre de la lista
def clasificar_calefaccion(calefaccion, listas):
    for tipo, lista in listas.items():
        if calefaccion in lista:
            return tipo
    return 'No clasif'

In [4622]:
def agrupar_cocinas(tipo):
    categorias = []
    if pd.isnull(tipo):
        categorias.append('sin_informacion')
    elif isinstance(tipo, str):
        if 'abierta' in tipo.lower() or 'abierto' in tipo.lower() or 'americana' in tipo.lower() or 'integrada' in tipo.lower():
            categorias.append('abierta')
        if 'independiente' in tipo.lower():
            categorias.append('independiente')
        if 'amplia' in tipo.lower():
            categorias.append('amplia')
        if 'isla' in tipo.lower():
            categorias.append('con isla')
        if 'amueblada' in tipo.lower():
            categorias.append('amueblada')
        if 'con electrodomesticos' in tipo.lower() or 'equipada' in tipo.lower() or 'con elec.' in tipo.lower():
            categorias.append('equipada')
        if 'sin amueblar' in tipo.lower():
            categorias.append('no amueblada')
        if 'reformada' in tipo.lower():
            return 'reformada'
        elif 'a reformar' in tipo.lower() or 'necesita reforma' in tipo.lower():
            return 'Sin reforma'
        if not categorias:
            categorias.append('sin categoria')
        return ', '.join(categorias)
    else:
        return 'error tipo campo no texto'
    return ', '.join(categorias)


In [4623]:

def clasificar_consumo(valor):
    try:
        valor = float(valor)
        if valor < 10:
            return 'A'
        elif valor <= 20:
            return 'B'
        elif valor <= 35:
            return 'C'
        elif valor <= 55:
            return 'D'
        elif valor <= 75:
            return 'E'
        elif valor <= 100:
            return 'F'
        else:
            return 'G'
    except ValueError:
        return None



In [4624]:
# Función para extraer solo el número de la cadena de texto
def extraer_numero(texto):
    if isinstance(texto, str):
        match = re.search(r'\d+', texto)
        return match.group(0) if match else None
    return None

# Carga de datos

In [4625]:

print(os.getcwd())

c:\Users\extas\Documents\GitHub\precios-de-vivienda-madrid\codigo\01-preprocesamiento-de-datos\Revision_columnas


In [4626]:
df17= pd.read_csv('./data/20250301_Union_todos.csv',sep=";",index_col=False)

C:\Users\extas\AppData\Local\Temp\ipykernel_21996\3324451358.py:1: DtypeWarning: Columns (3,12,13,14,17,20,23,34,51,53,54,57,58,63,64,69,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df17= pd.read_csv('./data/20250301_Union_todos.csv',sep=";",index_col=False)


In [4627]:
dfbarrios= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')

In [4628]:
# Reemplazar los valores 'na' por NaN
df17.replace('na', pd.NA, inplace=True)

In [4629]:
df17.columns

Index(['title', 'url', 'precio_anterior', 'descuento', 'EUR/m2', 'm2_constr',
       'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona', 'calle',
       'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia', 'estado',
       'amueblado', 'planta', 'dormitorios', 'banos', 'balcon', 'terraza',
       'garaje', 'trastero', 'ascensor', 'calefaccion', 'antiguedad',
       'num_plantas', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'nuda_propiedad', 'tipo_inmueble',
       'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpinteria_exterior_vidrio/metal', 'chimenea', 'cocina',
       'fibra_optica', 'exterior', 'interi

In [4630]:
df17.shape

(10205, 89)

In [4631]:
# Chqueo de nulos en todas las columnas
df17.isna().sum()
# Muestreo estadistico de las columnas

title                    0
url                      0
precio_anterior      10150
descuento            10150
EUR/m2                2350
                     ...  
letrace              10203
portero              10006
orientacion_este     10204
orientacion_oeste    10204
orientacion_sur      10202
Length: 89, dtype: int64

In [4632]:
df17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10205 entries, 0 to 10204
Data columns (total 89 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   title                                      10205 non-null  object 
 1   url                                        10205 non-null  object 
 2   precio_anterior                            55 non-null     float64
 3   descuento                                  55 non-null     object 
 4   EUR/m2                                     7855 non-null   float64
 5   m2_constr                                  10198 non-null  float64
 6   distrito                                   10198 non-null  object 
 7   cod_distrito                               10198 non-null  float64
 8   barrio                                     10137 non-null  object 
 9   cod_barrio                                 10137 non-null  float64
 10  zona                  

In [4633]:
df17.describe().T

,count,mean,std,min,25%,50%,75%,max
precio_anterior,55.00,830694.55,977555.73,98100.00,201500.00,440000.00,955000.00,4900000.00
EUR/m2,7855.00,3745.76,3996.23,0.00,7.08,3020.00,6360.00,27708.00
m2_constr,10198.00,138.65,150.97,0.00,68.00,99.00,150.00,4574.00
cod_distrito,10198.00,7.78,5.77,1.00,4.00,6.00,12.00,21.00
cod_barrio,10137.00,80.95,57.73,1.00,36.00,64.00,123.00,215.00
dormitorios,9976.00,2.83,1.42,0.00,2.00,3.00,3.00,24.00
balcon,1461.00,0.58,0.49,0.00,0.00,1.00,1.00,1.00
terraza,2499.00,0.77,0.42,0.00,1.00,1.00,1.00,1.00
trastero,2724.00,0.50,0.50,0.00,0.00,1.00,1.00,1.00
ascensor,4638.00,0.91,0.29,0.00,1.00,1.00,1.00,1.00


In [4634]:
df17["url"].isna().sum()

0

## m2_constr

In [4635]:
# Analisis descriptivo de la columna m2/constr
df17["m2_constr"].describe()

count   10198.00
mean      138.65
std       150.97
min         0.00
25%        68.00
50%        99.00
75%       150.00
max      4574.00
Name: m2_constr, dtype: float64

In [4636]:

#Filtro de valores menores a 20 metros cuadrados. Se analizo con 10 tambien y siguen siendo mas de 200
df17[df17["m2_constr"] <= 10].count()

title                3
url                  3
precio_anterior      0
descuento            0
EUR/m2               1
                    ..
letrace              0
portero              0
orientacion_este     0
orientacion_oeste    0
orientacion_sur      0
Length: 89, dtype: int64

#en la web ponen que tiene 10 m2, y por las fotos lo parece

In [4637]:
#voy a mirar los valores de m2_constr>1000
df17.query("m2_constr>= 4000")[['url','dormitorios','m2_constr']]

,url,dormitorios,m2_constr
2394,https://www.indomio.es/anuncios/96765659/,10.00,4574.00


In [4638]:
# Eliminar la fila donde la columna 'url' es igual a la URL especificada
df17 = df17[df17["url"] != "https://www.indomio.es/anuncios/96246263/"]


In [4639]:
# Resetear el índice del DataFrame
df17.reset_index(drop=True, inplace=True)

In [4640]:
#en la informacion dela web https://www.pisos.com/comprar/chalet-moncloa_aravaca_aravaca28023-4710772760_109700/ se dice 800 m2

# Actualizar el valor de m2_contr para la fila cuya URL contiene la cadena proporcionada
url_especifica = 'https://www.pisos.com/comprar/chalet-moncloa_aravaca_aravaca28023-'
#dfEDA.loc[dfEDA['url'].str.contains(url_especifica), ['m2_constr','dormitorios','banos','tiene_piscina',"num_garajes","trastero","estado"] = [800,5,5,1,2,1,'Obra nueva']
df17.loc[df17['url'].str.contains(url_especifica), ['m2_constr', 'dormitorios', 'banos', 'tiene_piscina', 'num_garajes', 'trastero', 'estado']] = [800, 5, 5, 1, 2, 1,'Obra nueva']
url_especifica='https://www.indomio.es/anuncios/95261315/'
df17.loc[df17['url'].str.contains(url_especifica), ['m2_constr',  'tiene_piscina', 'num_garajes', 'trastero', 'estado']] = [271,1,0, 0,'Buen Estado']
url_especifica='https://www.indomio.es/anuncios/94926741/'
df17.loc[df17['url'].str.contains(url_especifica), ['nuda_propiedad',  'tiene_piscina', 'num_garajes', 'trastero']] = [1,0,0,0]
#Elimino https://www.indomio.es/anuncios/91050501/ por que es un edificio con 5 estudios, no tenemos metros
url_a_eliminar = 'https://www.indomio.es/anuncios/91050501/'
df17 = df17[df17['url'] != url_a_eliminar]
df17.loc[df17["m2_constr"] == "3 baños", "m2_constr"] = np.nan

# se elimina la url https://www.pisos.com/comprar/piso-fontarron-47524209698_100600/ por que es un restaurante en venta



In [4641]:
df17.reset_index(drop=True, inplace=True)

In [4642]:
#quedan 5 viviendas sin m2_constr

#### distrito

In [4643]:
df17['distrito'].value_counts(dropna=False)

distrito
Centro                 1808
Salamanca              1532
Chamartín               640
Chamberí                612
Carabanchel             541
Puente De Vallecas      530
Tetuán                  467
Fuencarral-El Pardo     432
Retiro                  428
Latina                  416
Ciudad Lineal           409
Moncloa-Aravaca         380
San Blas-Canillejas     351
Arganzuela              298
Hortaleza               291
Villaverde              290
Villa De Vallecas       233
Usera                   196
Vicálvaro               137
Moratalaz               105
Barajas                 100
NaN                       7
Name: count, dtype: int64

In [4644]:
df17["distrito"].unique()

array(['Salamanca', 'Barajas', 'Arganzuela', 'Chamartín', 'Tetuán',
       'Chamberí', 'Villaverde', 'Latina', 'Centro',
       'Fuencarral-El Pardo', 'Villa De Vallecas', 'Puente De Vallecas',
       'Retiro', 'San Blas-Canillejas', 'Ciudad Lineal', 'Hortaleza',
       'Moncloa-Aravaca', 'Carabanchel', 'Vicálvaro', 'Usera',
       'Moratalaz', nan], dtype=object)

In [4645]:
# Ajustar la configuración para mostrar más caracteres en las columnas
pd.set_option('display.max_colwidth', None)

### nuda propiedad son imuebles sin posesion, se estan incluyendo inmuebles que se tiene la posesion pero no el uso y disfrute por que alguien tiene el usufructo, tabien se incluye inmuebles ocupados

In [4646]:
df17.query("cod_distrito.isnull()")[['distrito','url','cod_distrito','barrio','cod_barrio','m2_constr','planta','dormitorios','banos','precio','estado','tipologia','terraza']]

,distrito,url,cod_distrito,barrio,cod_barrio,m2_constr,planta,dormitorios,banos,precio,estado,tipologia,terraza
9391,NaN,https://www.pisosmadrid.com.es/propiedad/3890778-piso-en-venta-en-madrid,NaN,NaN,NaN,26.00,NaN,0.00,0.00,79900,NaN,Piso,NaN
9584,NaN,https://www.pisosmadrid.com.es/propiedad/3961341-piso-en-venta-en-madrid,NaN,NaN,NaN,109.00,NaN,1.00,1.00,138600,NaN,Piso,NaN
9592,NaN,https://www.pisosmadrid.com.es/propiedad/3963177-piso-en-venta-en-arturo-soria-madrid,NaN,NaN,NaN,70.00,NaN,1.00,1.00,450410,NaN,Piso,NaN
9680,NaN,https://www.pisosmadrid.com.es/propiedad/3998979-piso-en-venta-en-vallecas-madrid,NaN,NaN,NaN,98.00,NaN,2.00,2.00,205000,NaN,Piso,NaN
9794,NaN,https://www.pisosmadrid.com.es/propiedad/4029531-piso-en-venta-en-madrid,NaN,NaN,NaN,81.00,NaN,3.00,1.00,295000,NaN,Piso,NaN
9839,NaN,https://www.pisosmadrid.com.es/propiedad/4042093-piso-en-venta-en-madrid,NaN,NaN,NaN,139.00,NaN,3.00,2.00,426000,NaN,Piso,NaN
9960,NaN,https://www.pisosmadrid.com.es/propiedad/4056566-piso-en-venta-en-madrid,NaN,NaN,NaN,73.00,NaN,2.00,2.00,395000,NaN,Piso,NaN


No tenemos forma de conocer la ubicacion de https://www.pisosmadrid.com.es/propiedad/3930386-piso-en-venta-en-madrid	, lo elimino del dataset

In [4647]:
df17.query("cod_barrio.isnull()").count()

title                68
url                  68
precio_anterior       0
descuento             0
EUR/m2               19
                     ..
portero               0
orientacion_este      0
orientacion_oeste     0
orientacion_sur       0
num_garajes           0
Length: 90, dtype: int64

In [4648]:
df17['num_visitas'].value_counts(dropna=False)

num_visitas
NaN        7885
3.00        149
4.00        139
2.00        126
5.00         94
           ... 
2175.00       1
414.00        1
2279.00       1
322.00        1
1923.00       1
Name: count, Length: 1103, dtype: int64

#### portero

In [4649]:
df17["portero"].value_counts(dropna=False)

portero
NaN                      10004
Portero todo el día        145
1.0                         41
1                            6
Portero media jornada        4
todo el dia                  2
media jornada                1
Name: count, dtype: int64

In [4650]:

df17['portero'] = df17['portero'].replace('1', 'Portero todo el día')
df17['portero'] = df17['portero'].replace('1.0', 'Portero todo el día')
df17['portero'] = df17['portero'].replace('todo el dia', 'Portero todo el día')
df17['portero'] = df17['portero'].replace('media jornada', 'Portero media jornada')

## gastos comunidad

In [4651]:

df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('Ningún gasto de comunidad', '0')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace(',00', '')
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('.', '').str.strip()
df17['gastos_comunidad'] = df17['gastos_comunidad'].str.replace('37,50', '38').str.strip()

##  terraza

In [4652]:
df17["terraza"].value_counts(dropna=False)

terraza
NaN     7705
1.00    1929
0.00     569
Name: count, dtype: int64

In [4653]:
df17['tiene_armario'] = df17['tiene_armario'].replace(0.0, np.nan)
df17['tiene_piscina'] = df17['tiene_piscina'].replace(0, np.nan)

# calefaccion

In [4654]:
df17["calefaccion"].value_counts(dropna=False)

calefaccion
NaN                                                   6724
Central                                                905
Individual                                             583
1.0                                                    565
Gas natural                                            520
 Gas Natural                                           251
 Eléctrica                                             123
Eléctrica                                              113
 Individual                                            104
 Central                                                87
Individual, alimentación eléctrica                      34
Individual, por radiadores, de gas                      30
Individual - gas natural                                18
Central, por radiadores, de gas                         13
Suelo radiante                                          11
Individual, por radiadores                              10
Gasoil                                      

In [4655]:


# Supongamos que df17 es tu DataFrame
#df17['portero'] = df17['portero'].replace('0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('0.0', np.nan)
df17['calefaccion'] = df17['calefaccion'].replace('Climatización', 'climatizacion')
df17['calefaccion'] = df17['calefaccion'].replace('Climatizaciòn', 'climatizacion')
df17['calefaccion']=df17['calefaccion'].str.strip()

In [4656]:


# Palabras clave para clasificar
central_keywords = ["central", "comunitaria", "comunitario"]
individual_keywords = ["individual", "particular", "chimenea", "bomba", "termo", "climatizacion"]
electric_keywords = ["electrica", "bomba", "climatización", 'termo electrico','climatizacion']
gas_keywords = ["gas", "propano", "g n", "g/n","G/n"]  # Asegurarse de que 'g n' y 'g/n' estén bien escritos
gasoil_keywords = ["gasóil","gasoil",'gasoil',"Gasoil"]  # Añadir "gasoil" aquí
renovables_keywords = ["aerotermia", "geotermia", "chimenea"]
suelo_radiante_keywords=["suelo_radiante", "suelo"]
#radiadores_keywords=["radiadores"]
aire_keywords=["bomba","climatización",'climatizacion','conducto']
desconocido_keywords=['chimenea', 'termo electrico']
#lo mas extendido son los radiadores por eso despues de pasar , suelo radiante y aire pongo a todos radiadores salvo a chimenea y termo electrico que no lo se
radiadores_keywords=['radiadores','Gas natural', 'G/n', 'Gas particular', 'Central, alimentación eléctrica', 'Agua caliente y calefacción central con contador i', 'Comunitario', 'Individual con gas natural', 'Individual, de gasóil', 'Comunitaria', 'Calefaccion Propano', 'Calefaccion Gas Natural', 'Central, por aire, de gas', 'Gas natural y biomasa', 'Centralizada', 'Eléctrica', 'Calefaccion Individual', 'Calefaccion individual','Contadores individuales', '1.0', 'Individual, alimentación eléctrica', 'Aerotermia', 'Individual - gas natural', 'Individual', 'Calefaccion Aerotermia', 'Gasoil', 'Central individualizada', 'Individual, por aire, alimentación eléctrica', 'Geotermia y aerotermia', 'Calefaccion comunitaria', 'Calefaccion Central', 'Central, de gas', 'Calefaccion Electrica', 'Calefacción central individualizada', 'Central, de gasóil', 'Individual, de gas', 'Central']

# Inicializar conjuntos para valores únicos
central = set()
individual = set()
electrica = set()
gas = set()
gsoil = set()
renovables = set()
suelo_radiante = set()
radiadores = set()
aire = set()
sin_clasificar = set()
sin_clasificar_energia = set()
sin_clasificar_medio = set()
radiadores=set()
suelo_radiante=set()


# Clasificar categorías
for categoria in df17["calefaccion"]:
    categoria_normalized = normalize_text(str(categoria))  # Normalizar texto

    # Clasificación por tipo (central o individual)
    if any(keyword in categoria_normalized for keyword in central_keywords):
        central.add(categoria)
    elif any(keyword in categoria_normalized for keyword in individual_keywords):
        individual.add(categoria)
    else:
        sin_clasificar.add(categoria)

    # Clasificación por energía (eléctrica, gas, gasoil)
    if any(keyword in categoria_normalized for keyword in electric_keywords):
        electrica.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gasoil_keywords):
        gsoil.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gas_keywords):
        gas.add(categoria)


    elif any(keyword in categoria_normalized for keyword in renovables_keywords):
        renovables.add(categoria)
    else:
        sin_clasificar_energia.add(categoria)

    # Clasificación por medio (suelo radiante, radiadores, aire)


    if any(keyword in categoria_normalized for keyword in aire_keywords):
        aire.add(categoria)
    elif any(keyword in categoria_normalized for keyword in suelo_radiante_keywords):
        suelo_radiante.add(categoria)
    elif any(keyword in categoria_normalized for keyword in desconocido_keywords):
        sin_clasificar_medio.add(categoria)
  #  else: any(keyword in categoria_normalized for keyword in radiadores_keywords):
    else:
        radiadores.add(categoria)


# Convertir los conjuntos a listas
central = list(central)
individual = list(individual)
sin_clasificar = list(sin_clasificar)
electrica = list(electrica)
gas = list(gas)
gasoil = list(gsoil)
renovables = list(renovables)
sin_clasificar_energia = list(sin_clasificar_energia)
aire=list(aire)
radiadores=list(radiadores)
suelo_radiante=list(suelo_radiante)
sin_clasificar_medio=list(sin_clasificar_medio)
# Imprimir resultados
print("Central:", central)
print("Individual:", individual)
print("No clasificado:", sin_clasificar)
print("Electrica:", electrica)
print("Gas:", gas)
print("Gasoil:", gasoil)
print("Renovables:", renovables)
print("No clasificado en energía:", sin_clasificar_energia)
print("Suelo Radiante:", suelo_radiante)
print("Radiadores:", radiadores)
print("Aire:", aire)
print("No clasificado en medio:", sin_clasificar_medio)


Central: ['Central, por aire, de gas', 'Calefaccion comunitaria', 'Central, suelo radiante, de gas', 'Central, de gas', 'Agua caliente y calefacción central con contador i', 'Central', 'Comunitario', 'Centralizada', 'Central, alimentación eléctrica', 'Central, por radiadores, de gasóil', 'Central, suelo radiante', 'Central, por radiadores, de gas', 'central por gas', 'Comunitaria', 'Central, de gasóil', 'Central, por radiadores', 'Central individualizada', 'central', 'Calefacción central individualizada']
Individual: ['Individual', 'individual gas natural', 'Individual, de gas', 'Termo electrico', 'Individual con gas natural', 'Bomba de calor', 'Climatizacion', 'Individual, suelo radiante, de gas', 'Individual, por radiadores', 'climatizacion', 'Bomba frio/calor', 'Bomba frio-calor', 'Por climatizacion', 'Contadores individuales', 'Individual, por radiadores, de gas', 'Individual, por radiadores, de gasóil', 'Individual, por radiadores, alimentación eléctrica', 'Individual, suelo radia

In [4657]:
df17["antiguedad"].value_counts(dropna=False)

antiguedad
NaN                    6762
Más de 50 años         1261
Entre 30 y 50 años      208
1960                    109
1900                    109
                       ... 
1916                      1
Entre 10 y  15 Años       1
 1970                     1
 1979                     1
2007.0                    1
Name: count, Length: 165, dtype: int64

In [4658]:
df17["num_plantas"].value_counts(dropna=False)

num_plantas
NaN           9970
6.0             40
4.0             39
3.0             28
7.0             24
5.0             23
8.0             19
10.0            13
2.0             13
Plantas: 5       6
9.0              6
Plantas: 4       5
1.0              5
Plantas: 6       3
Plantas: 3       2
12.0             1
11.0             1
23.0             1
Plantas: 2       1
Plantas: 7       1
Plantas: 1       1
Plantas: 8       1
Name: count, dtype: int64

In [4659]:
df17["num_plantas"]=df17["num_plantas"].str.replace("Plantas: ","",regex=False)

## aire acondicionado

In [4660]:
df17["aire_acondicionado"].value_counts(dropna=False)

aire_acondicionado
NaN                                                   7642
Individual                                             583
Frío                                                   480
1.0                                                    465
Individual, frío/calor                                 399
Frío y calor                                           320
Individual, frío                                       115
Splites repartidos                                      70
Independiente                                           26
Frío/calor                                              20
Centralizada                                            16
Suelo refrigerante                                      10
Frío-calor                                               7
Preinstalación                                           6
Preinstalado                                             5
Aerotermia                                               5
Central                              

In [4661]:
df17["aire_acondicionado"].unique()


array([nan, 'Preinstalación', 'Individual, frío/calor',
       'Individual, frío', 'Individual', 'Frío/calor',
       'Centralizada, frío', 'Centralizada, frío/calor',
       'Centralizada, calor', 'Centralizada', 'Frío', 'Frío y calor',
       'Splites repartidos', 'Aerotermia', 'A/a f/c',
       'Con bomba de calor', 'Preinstalado', 'Independiente',
       'Suelo refrigerante', 'Frío-calor', 'Sistema air zone',
       'Geotermia y aerotermia',
       'Climatizacion por conductos frio y calor',
       'Tiene la instalción hecha en toda la casa por cond',
       'Suelo radiante refrescante', 'No disponible', 'Central',
       'Frió/calor por conductos', 'Suelo radiante',
       'En toda la casa, por conductos',
       'Aire frio/ calor por conductos/suelo refrescante',
       'Por conductos en toda la casa', '1.0', 'por splits', '1',
       'bomba frio calor'], dtype=object)

In [4662]:
# Listas de valores para cada grupo
eficientes_val_columna = [
    'Aerotermia', 'Geotermia y aerotermia', 'Suelo radiante refrescante', 'Suelo refrigerante',
    'Sistema air zone', 'Climatizacion por conductos frio y calor', 'Frió/calor por conductos',
    'Suelo radiante', 'Aire frio/ calor por conductos/suelo refrescante', 'Por conductos en toda la casa',
    'Tiene la instalción hecha en toda la casa por cond', 'En toda la casa, por conductos'
]

normales_val_columna = [
    'Individual, frío/calor', 'Individual, frío', 'Individual', 'Frío/calor', 'Centralizada, frío',
    'Centralizada, frío/calor', 'Centralizada, calor', 'Central', 'Frío', 'Frío y calor',
    'Splites repartidos', 'Independiente', 'Con bomba de calor', 'A/a f/c',
    'Aire frio/calor por conductos/suelo refrescante', '1', '1.0'
]

sin_aire_val_columna = ['Preinstalación', 'Preinstalado', 'No disponible', '0', '0.0']

# Función para clasificar los sistemas
def clasificar_sistema(valor):
    if valor in eficientes_val_columna:
        return 'Eficiente'
    elif valor in normales_val_columna:
        return 'Normal'
    elif valor in sin_aire_val_columna:
        return 'Sin Aire'
    else:
        return 'Desconocido'  # Si el valor no coincide con ninguno de los grupos




In [4663]:
# Crear una nueva columna en el DataFrame
df17['clasificacion_aire'] = df17['aire_acondicionado'].apply(clasificar_sistema)


La columna disponibilidad no aporta nada

In [4664]:
df17["lujosa"].value_counts(dropna=False)

lujosa
NaN     7060
0.00    1715
1.00    1428
Name: count, dtype: int64

In [4665]:


# Aplicar la lógica: si 'lujosa' es 1 y 'tipo_inmueble' es nulo (NaN), entonces 'tipo_inmueble' = 'lujo'
df17['tipo_inmueble'] = df17.apply(
    lambda fila: 'lujo' if fila['lujosa'] == 1.00 and pd.isna(fila['tipo_inmueble']) else fila['tipo_inmueble'], axis=1
)



In [4666]:
df17=df17.drop(columns='lujosa')

In [4667]:
df17.columns

Index(['title', 'url', 'precio_anterior', 'descuento', 'EUR/m2', 'm2_constr',
       'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona', 'calle',
       'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia', 'estado',
       'amueblado', 'planta', 'dormitorios', 'banos', 'balcon', 'terraza',
       'garaje', 'trastero', 'ascensor', 'calefaccion', 'antiguedad',
       'num_plantas', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'nuda_propiedad', 'tipo_inmueble',
       'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpinteria_exterior_vidrio/metal', 'chimenea', 'cocina',
       'fibra_optica', 'exterior', 'interi

In [4668]:
df17['tiene_armario'].value_counts(dropna=False)

tiene_armario
NaN     8401
1.00    1802
Name: count, dtype: int64

In [4669]:
df17['armarios'].value_counts(dropna=False)

armarios
 NaN                      8477
 1                         810
 Más de 2                  254
2.00                       143
1.00                       120
3.00                       104
 2                          71
4.00                        70
 1.0                        42
 2.0                        24
5.00                        23
6.00                        18
 3.0                        17
 5.0                         6
 4.0                         6
7.00                         5
8.00                         3
 8.0                         3
 6.0                         1
 3 empotrados                1
 8 armarios empotrados       1
 4 empotrados                1
 Mas de 5                    1
 1 closet                    1
9.00                         1
Name: count, dtype: int64

In [4670]:

# Lista de palabras a reemplazar
palabras_a_eliminar = ["empotrados", "armarios", "closet"]

# Crear una expresión regular que abarque todas las palabras
patron = '|'.join(palabras_a_eliminar)

# Reemplazar las palabras por vacío
df17['armarios'] = df17['armarios'].str.replace(patron, '', regex=True)




In [4671]:
# Aplicar la lógica: si 'armarios' no es NaN, entonces 'tiene_armario' = 1
df17['tiene_armario'] = df17.apply(
    lambda fila: 1 if pd.notna(fila['armarios']) and fila['armarios'] != '' else fila['tiene_armario'], axis=1
)

In [4672]:
df17['cancha_tenis'].value_counts(dropna=False)

cancha_tenis
NaN     7061
0.00    3140
1.00       2
Name: count, dtype: int64

In [4673]:
df17['chimenea'].value_counts(dropna=False)

chimenea
NaN     7057
0.00    3097
1.00      49
Name: count, dtype: int64

In [4674]:
df17['fibra_optica'].value_counts(dropna=False)

fibra_optica
NaN     7061
0.00    3104
1.00      38
Name: count, dtype: int64

In [4675]:
df17['carpinteria_exterior_doble_vidrio/pvc'].value_counts(dropna=False)

carpinteria_exterior_doble_vidrio/pvc
NaN     6703
0.00    3126
1.00     374
Name: count, dtype: int64

## cocina

In [4676]:
df17['cocina'].value_counts(dropna=False)

cocina
NaN                                                 9160
Cocina independiente                                 323
Cocina amueblada                                     190
Independiente                                        131
Cocina abierta                                        72
                                                    ... 
Con tendedero que se puede incluir en la cocicna       1
Grande                                                 1
Sin amueblar                                           1
Amplia y completamente amueblada                       1
amueblada equipada                                     1
Name: count, Length: 96, dtype: int64

In [4677]:
df17['cocina'].unique()

array([nan, 'Cocina abierta', 'Cocina independiente',
       'Independiente y equipada', 'Cocina americana', 'Cocina equipada',
       'Cocina amueblada', 'Amueblada sin electrodomésticos',
       'Amueblada con electrodomésticos', 'Americana',
       'Independiente con zona de servicio',
       'Independiente - amueblada con elec.', 'Independiente',
       'Independiente con office', 'Amueblada', 'Equipada', 'Amplia',
       'Amueblada y equipada con electrodomésticos',
       'Pequeña cocina integrada', 'Americana, amueblada y equipada.',
       'Gran cocina abierta con isla', 'Moderna', 'Individual',
       'Hall de entrada', 'Reformada', 'Completamente equipada',
       'Totalmente equipada',
       'Abierta al comedor, completamente equipada',
       'En office amplia completamente equipada',
       'Cocina amueblada, sin aparatos', 'Con tendedero', 'Con terraza',
       'Dos cocinas, una en cada planta',
       'Amueblada con electrodomésticos, caldera nueva',
       'No incluye 

In [4678]:
df17.query("amueblado=='Sólo cocina amueblada' and cocina.notnull()")[['amueblado','cocina']]

,amueblado,cocina
492,Sólo cocina amueblada,Cocina abierta
1279,Sólo cocina amueblada,Cocina independiente
1748,Sólo cocina amueblada,Cocina independiente
1799,Sólo cocina amueblada,Cocina independiente
2823,Sólo cocina amueblada,Cocina independiente


In [4679]:

df17.loc[df17['cocina']=='Francesa', 'cocina'] = 'independiente y amueblada'
df17.loc[(df17['amueblado'] == 'Sólo cocina amueblada') & (df17['cocina'].notnull()), 'cocina'] = df17['cocina'] + ' amueblada'
df17.loc[df17['cocina']=='Equipafda americana', 'cocina'] = 'Equipada americana'



In [4680]:
# Aplica la función de agrupación
df17['grupo_cocina'] = df17['cocina'].apply(agrupar_cocinas)

print(df17[['cocina', 'grupo_cocina']])


               cocina     grupo_cocina
0                 NaN  sin_informacion
1      Cocina abierta          abierta
2                 NaN  sin_informacion
3                 NaN  sin_informacion
4                 NaN  sin_informacion
...               ...              ...
10198             NaN  sin_informacion
10199             NaN  sin_informacion
10200             NaN  sin_informacion
10201             NaN  sin_informacion
10202             NaN  sin_informacion

[10203 rows x 2 columns]


In [4681]:
df17.query("cocina.notnull()")['cocina'].unique()

array(['Cocina abierta', 'Cocina independiente',
       'Cocina abierta amueblada', 'Cocina independiente amueblada',
       'Independiente y equipada', 'Cocina americana', 'Cocina equipada',
       'Cocina amueblada', 'Amueblada sin electrodomésticos',
       'Amueblada con electrodomésticos', 'Americana',
       'Independiente con zona de servicio',
       'Independiente - amueblada con elec.', 'Independiente',
       'Independiente con office', 'Amueblada', 'Equipada', 'Amplia',
       'Amueblada y equipada con electrodomésticos',
       'Pequeña cocina integrada', 'Americana, amueblada y equipada.',
       'Gran cocina abierta con isla', 'Moderna', 'Individual',
       'Hall de entrada', 'Reformada', 'Completamente equipada',
       'Totalmente equipada',
       'Abierta al comedor, completamente equipada',
       'En office amplia completamente equipada',
       'Cocina amueblada, sin aparatos', 'Con tendedero', 'Con terraza',
       'Dos cocinas, una en cada planta',
       'Amue

In [4682]:
df17.query("amueblado=='Sólo cocina amueblada'").count()

title                 69
url                   69
precio_anterior        0
descuento              0
EUR/m2                68
                      ..
orientacion_oeste      0
orientacion_sur        0
num_garajes            1
clasificacion_aire    69
grupo_cocina          69
Length: 91, dtype: int64

In [4683]:
df17['amueblado'].value_counts(dropna=False)

amueblado
 NaN                       8955
 1.0                        575
 Sí                         274
1.00                        159
 No                         158
 Sólo cocina amueblada       69
 Parcialmente amueblado      13
Name: count, dtype: int64

In [4684]:
## interior, exterior
df17['interior'].value_counts(dropna=False)

interior
NaN     7059
0.00    3123
1.00      21
Name: count, dtype: int64

In [4685]:

df17['exterior'].value_counts(dropna=False)

exterior
NaN     4628
0.00    3008
1.00    2567
Name: count, dtype: int64

In [4686]:

df17['interior_y_exterior'].value_counts(dropna=False)

interior_y_exterior
NaN     7061
0.00    3134
1.00       8
Name: count, dtype: int64

In [4687]:

df17.query("interior.isnull() & exterior.isnull() & interior_y_exterior.isnull()")["web"].unique()

array(['pisos.com', 'redpiso', 'pisosmadrid'], dtype=object)

## piscina

In [4688]:


df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     10143
1.00       60
Name: count, dtype: int64

In [4689]:
df17.loc[df17['url'] == 'https://www.indomio.es/anuncios/95261315/', 'piscina'] = 'comunitaria'

In [4690]:
df17['piscina'].value_counts(dropna=False)

piscina
 NaN                                          5986
 0                                            2744
0.00                                           750
 1                                             397
 Comunitaria                                   139
1.00                                           108
 Propia                                         76
 comunitaria                                     1
 1 piscina de adultos y 1 piscina de niños       1
 Con salorium                                    1
Name: count, dtype: int64

In [4691]:
df17['piscina'] = df17['piscina'].astype(str).str.lower().apply(unidecode).str.strip().replace('nan', None)

In [4692]:
df17['piscina']=df17['piscina'].replace('0.0','sin piscina')
df17['piscina']=df17['piscina'].replace('0','sin piscina')

In [4693]:
df17['piscina'].value_counts(dropna=False)

piscina
None                                         5986
sin piscina                                  3494
1                                             397
comunitaria                                   140
1.0                                           108
propia                                         76
1 piscina de adultos y 1 piscina de ninos       1
con salorium                                    1
Name: count, dtype: int64

In [4694]:
# Aplicar la lógica solo a los valores nulos en 'tiene_jardin'
df17['tiene_piscina'] = df17['tiene_piscina'].where(
    pd.notnull(df17['tiene_piscina']),  # Mantener valores no nulos existentes
    df17['piscina'].apply(lambda x: 1 if pd.notnull(x) and x != 'sin piscina' else (0 if x == 'sin piscina' else None))
)



In [4695]:
df17['tiene_piscina'].value_counts(dropna=False)

tiene_piscina
NaN     5933
0.00    3491
1.00     779
Name: count, dtype: int64

In [4696]:
# Aplicar condiciones a la columna 'piscina'
df17['piscina'] = df17.apply(
    lambda x: 'privada' if pd.isnull(x['piscina']) and x['tiene_piscina'] == 1 and x['tipologia'] == 'Chalet' else
              ('comunitaria' if pd.isnull(x['piscina']) and x['tiene_piscina'] == 1 and x['tipologia'] != 'Chalet' else x['piscina']),
    axis=1
)



In [4697]:
df17['piscina'].value_counts(dropna=False)

piscina
None                                         5933
sin piscina                                  3494
1                                             397
comunitaria                                   184
1.0                                           108
propia                                         76
privada                                         9
1 piscina de adultos y 1 piscina de ninos       1
con salorium                                    1
Name: count, dtype: int64

In [4698]:
# Actualizar la columna 'tiene_piscina' donde 'piscina' es igual a 'sin piscina'
df17.loc[df17['piscina'] == 'sin piscina', 'tiene_piscina'] = 0




In [4699]:
df17.query("piscina=='sin piscina' & tiene_piscina==1")

,title,url,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letra_ce,emisiones_co2,tipologia,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,calefaccion,antiguedad,num_plantas,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,nuda_propiedad,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,puerta_blindada,alarma,videoportero,€_comunidad_mes,acceso_discapacitado,orientacion,precio,web,fecha_descarga,actualizacion,armarios,codigo_Postal,m2_utiles,referencia,tipo_suelo,certificado_energetico,tiene_jardin,descripcion_larga,num_visitas,fecha_publicacion,titularanuncio,num_pisos,puerta_seguridad,zonas_verdes,gastos_comunidad,tiene_piscina,tipo_fachada,agua_caliente,consumo_energia,nombre_oficina,letrace,portero,orientacion_este,orientacion_oeste,orientacion_sur,num_garajes,clasificacion_aire,grupo_cocina


## porton electrico, puerta blindada

In [4700]:

## portn_electrico,puerta_blindad

columnas = ["porton_electrico", "puerta_blindada", "alarma","videoportero","descripcion_larga"]
porcentaje_resultados = porcentaje_nulos(df17, columnas)


# Mostrar los resultados
for columna, porcentaje in porcentaje_resultados.items():
    print(f"Porcentaje de nulos en {columna}: {porcentaje:.2f}%")

Porcentaje de nulos en porton_electrico: 69.21%
Porcentaje de nulos en puerta_blindada: 69.13%
Porcentaje de nulos en alarma: 68.23%
Porcentaje de nulos en videoportero: 66.44%
Porcentaje de nulos en descripcion_larga: 18.88%


In [4701]:

df17['porton_electrico'].value_counts(dropna=False)

porton_electrico
NaN     7061
0.00    3126
1.00      16
Name: count, dtype: int64

In [4702]:

df17['puerta_blindada'].value_counts(dropna=False)

puerta_blindada
NaN                                                   7053
0                                                     3050
1                                                       92
Acorazada                                                2
Tiene 3 puertas de acceso                                1
Muy cerca centro comercial las rosas, metro y bus.       1
Puerta acorazada                                         1
Bombillo seguridad multilock                             1
Puerta a la calle                                        1
Puerta de entrada blindada                               1
Name: count, dtype: int64

In [4703]:

df17['alarma'].value_counts(dropna=False)

alarma
 NaN                                              6961
 0                                                3100
1.00                                                48
 1                                                  42
 Portero físico                                     12
 vigilancia 24h, con cámaras de seguridad            5
 Conserje                                            4
 con cámaras de seguridad                            3
 Servicio de portería                                2
 Alarma                                              2
 Portero fisico                                      2
 24h                                                 2
 24 horas                                            2
 Alarma y cámaras                                    1
 Alarma instalada.                                   1
 Conserje 24 horas                                   1
 Circuito de video vigilancia en zonas comunes       1
 Cámaras de vigilancia en portal                     1
 Po

In [4704]:
# Actualizar la columna 'portero' únicamente donde 'alarma' contiene "portero"
df17['portero'] = df17.apply(
    lambda fila: 1 if "portero" in str(fila['alarma']).lower() else fila['portero'], axis=1
)

df17['portero'] = df17.apply(
    lambda fila: 1 if "conserje" in str(fila['alarma']).lower() else fila['portero'], axis=1
)

df17['portero'] = df17.apply(
    lambda fila: 1 if "vigilancia" in str(fila['alarma']).lower() else fila['portero'], axis=1
)

df17['alarma'] = df17.apply(
    lambda fila: 1 if "alarma" in str(fila['alarma']).lower() else fila['alarma'], axis=1
)

In [4705]:
df17.columns

Index(['title', 'url', 'precio_anterior', 'descuento', 'EUR/m2', 'm2_constr',
       'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona', 'calle',
       'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia', 'estado',
       'amueblado', 'planta', 'dormitorios', 'banos', 'balcon', 'terraza',
       'garaje', 'trastero', 'ascensor', 'calefaccion', 'antiguedad',
       'num_plantas', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'nuda_propiedad', 'tipo_inmueble',
       'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpinteria_exterior_vidrio/metal', 'chimenea', 'cocina',
       'fibra_optica', 'exterior', 'interi

In [4706]:

df17.query("alarma==1 | puerta_blindada==1")[['precio','alarma','puerta_blindada']]

,precio,alarma,puerta_blindada
3373,1050000,1,NaN
3374,1985000,1,NaN
4331,225000,1,NaN
5869,145000,1,NaN
8715,81500,1.00,NaN
8754,115000,1.00,NaN
9297,925000,1.00,NaN
9351,419900,1.00,NaN
9376,229000,1.00,NaN
9399,450000,1.00,NaN


€_comunidad_mes

In [4707]:
df17['€_comunidad_mes'].value_counts(dropna=False)

€_comunidad_mes
NaN    9676
50       23
60       22
100      17
70       15
       ... 
270       1
9         1
39        1
294       1
108       1
Name: count, Length: 204, dtype: int64

In [4708]:
df17['gastos_comunidad'].value_counts(dropna=False)

gastos_comunidad
NaN     10169
50          8
45          4
40          3
69          1
2110        1
500         1
550         1
380         1
60          1
236         1
38          1
30          1
35          1
150         1
74          1
65          1
110         1
100         1
180         1
20          1
99          1
850         1
Name: count, dtype: int64

In [4709]:
df17['gastos_comunidad'] = df17.apply(
    lambda fila: fila['€_comunidad_mes'] if pd.isna(fila['gastos_comunidad']) else fila['gastos_comunidad'], axis=1
)

In [4710]:
df17=df17.drop(columns='€_comunidad_mes')

In [4711]:
df17.shape

(10203, 90)

In [4712]:
df17['acceso_discapacitado'].value_counts(dropna=False)

acceso_discapacitado
 NaN                              9699
 Exteriores                        211
 0.0                               187
 1.0                                69
 Ver todas las características      25
1.00                                10
 Otros                               2
Name: count, dtype: int64

## acceso_discapacitados

In [4713]:

# Renombrar la columna 'acceso_discapacitado' a 'acceso_discapacidad'
df17.rename(columns={'acceso_discapacitado': 'acceso_discapacidad'}, inplace=True)

## orientacion

In [4714]:
df17["orientacion"].value_counts()

orientacion
Sur                                312
Este                               263
 Sur                               161
 Este                              150
Oeste                              148
Norte                              131
 Oeste                             125
Sureste                            105
 Norte                              80
 Sureste                            71
Suroeste                            59
 Suroeste                           59
 Noroeste                           53
Noreste                             52
Noroeste                            51
 Noreste                            50
sur                                 11
Medio día                           10
este                                 9
-1                                   9
norte                                4
Nordeste                             2
oeste                                2
A la montaña                         2
sur_este                             2
Sur en la ter

In [4715]:
# Eliminar 'Orientacion' y limpiar espacios adicionales
df17['orientacion'] = df17['orientacion'].str.replace('Orientacion', '').str.strip()

# Sustituir ',' y '-' por '_' y poner los valores en minúsculas en la columna 'orientacion'
df17['orientacion'] = df17['orientacion'].str.replace(',', '_').str.replace('/', '_').str.replace('|', '_').str.replace('y', '_').str.replace('-', '_').str.lower()

# Eliminar espacios adicionales entre palabras
df17['orientacion'] = df17['orientacion'].str.replace(r'\s+', '_', regex=True).str.strip()
df17['orientacion'] = df17['orientacion'].str.replace('__', '_')

# Reemplazar múltiples valores en la columna 'orientacion' por np.nan
df17['orientacion'] = df17['orientacion'].replace({
    '__':'_',
    '_1': np.nan,
    'medio_día': np.nan,
    'sur_en_la_terraza__dormitorios': np.nan,
    'vistas_al_palacio_de_parcent.': np.nan,
    'a_la_montaña': np.nan,
    'varias':np.nan
})

df17['orientacion'] = df17['orientacion'].str.replace('__', '_')



In [4716]:
# Obtener los valores únicos de la columna 'orientacion' donde los valores no son nulos
df17.loc[df17['orientacion'].notnull(), 'orientacion'].unique()


array(['norte', 'sur_este', 'este', 'oeste', 'sur', 'norte_sur',
       'suroeste', 'sureste', 'noroeste', 'este_oeste', 'noreste',
       'sur_este_norte', 'este_noreste_suroeste', 'oeste_este',
       'norte_este', 'este_oeste_norte', 'sur_oeste', 'sudeste',
       'sur_norte', 'nordeste'], dtype=object)

In [4717]:


# Procesar la columna 'orientacion'
for index, value in df17["orientacion"].items():
    # hasta los nan los convierte a string, recordar luego cambiar 'nan' por nan de nuevo
    value_str = str(value)
    if re.search(r'\bnorte\b', value_str):
        df17.at[index, "orientacion_norte"] = 1
    if re.search(r'\bsur\b', value_str):
        df17.at[index, "orientacion_sur"] = 1
    if re.search(r'\beste\b', value_str):
        df17.at[index, "orientacion_este"] = 1
    if re.search(r'\boeste\b', value_str):
        df17.at[index, "orientacion_oeste"] = 1
    if re.search(r'\bnoroeste\b', value_str):
        df17.at[index, "orientacion_norte"] = 1
        df17.at[index, "orientacion_oeste"] = 1
    if re.search(r'\bsuroeste\b', value_str):
        df17.at[index, "orientacion_sur"] = 1
        df17.at[index, "orientacion_oeste"] = 1
    if re.search(r'\bsureste\b', value_str):
        df17.at[index, "orientacion_sur"] = 1
        df17.at[index, "orientacion_este"] = 1
    if re.search(r'\bnordeste\b', value_str):
        df17.at[index, "orientacion_norte"] = 1
        df17.at[index, "orientacion_este"] = 1
    if re.search(r'\bsur_este\b', value_str):
        df17.at[index, "orientacion_sur"] = 1
        df17.at[index, "orientacion_este"] = 1


        

# Revisar las columnas orientacion_norte,_sur y orientacion_sur,_este
# y asegurarnos de que las combinaciones sean coherentes
if "orientacion_norte,_sur" in df17.columns:
    for index, value in df17["orientacion_norte,_sur"].items():
        if value == 1:
            df17.at[index, "orientacion_norte"] = 1
            df17.at[index, "orientacion_sur"] = 1

if "orientacion_sur,_este" in df17.columns:
    for index, value in df17["orientacion_sur,_este"].items():
        if value == 1:
            df17.at[index, "orientacion_sur"] = 1
            df17.at[index, "orientacion_este"] = 1


In [4718]:
# Filtrar filas donde todas las columnas relevantes no sean nulas
filtered_df = df17[
    df17["orientacion"].notnull() |
    df17["orientacion_norte"].notnull() |
    df17["orientacion_sur"].notnull() |
    df17["orientacion_este"].notnull() |
    df17["orientacion_oeste"].notnull()
   
]

# Mostrar las columnas deseadas
result = filtered_df[["orientacion", "orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]]
print(result)


      orientacion  orientacion_norte  orientacion_sur  orientacion_este  \
93          norte               1.00              NaN               NaN   
174      sur_este                NaN             1.00              1.00   
228          este                NaN              NaN              1.00   
310      sur_este                NaN             1.00              1.00   
361         oeste                NaN              NaN               NaN   
...           ...                ...              ...               ...   
9762          NaN                NaN              NaN              1.00   
9874          NaN                NaN              NaN               NaN   
9907          NaN                NaN             1.00               NaN   
9908          NaN                NaN             1.00               NaN   
10103         NaN                NaN             1.00               NaN   

       orientacion_oeste  
93                   NaN  
174                  NaN  
228               

In [4719]:
#eliminacion de las columnas orientacion,orientacion_sur,_este,orientacion_norte,_sur
df17.drop(columns=['orientacion'], inplace=True)


In [4720]:
df17.query("orientacion_norte!=1 and orientacion_sur!=1  and orientacion_este!=1  and orientacion_oeste!=1 ").count()

title                 8400
url                   8400
precio_anterior         55
descuento               55
EUR/m2                6755
                      ... 
orientacion_sur          0
num_garajes             12
clasificacion_aire    8400
grupo_cocina          8400
orientacion_norte        0
Length: 90, dtype: int64

In [4721]:
# Poner NaN en las columnas si no cumplen con la condición
df17.loc[df17.query("orientacion_norte!=1 and orientacion_sur!=1 and orientacion_este!=1 and orientacion_oeste!=1").index, ["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]] = np.nan

# Asegurarse de que si alguna columna tiene valor 1, las demás sean 0
df17[["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]] = df17[["orientacion_norte", "orientacion_sur", "orientacion_este", "orientacion_oeste"]].fillna(0)

## web

In [4722]:
df17["web"].value_counts(dropna=False)

web
pisos.com      4728
Indomio.com    3142
redpiso        1460
pisosmadrid     873
Name: count, dtype: int64

In [4723]:
df17["fecha_descarga"].value_counts(dropna=False)

fecha_descarga
2024-11-20    3142
19/08/2024    2745
19/10/2024    1983
28/10/2024    1460
24/10/2024     873
Name: count, dtype: int64

## certificado_energetico

In [4724]:
df17["certificado_energetico"].value_counts(dropna=False)

certificado_energetico
NaN            4196
0.0            2254
No indicado    1832
Disponible     1310
1.0             611
Name: count, dtype: int64

In [4725]:
df17.query("consumoce_ano.notnull() | letrace.notnull() | emisiones_co2.notnull()" )[ ["certificado_energetico",'consumoce_ano','letrace','emisiones_co2']]

,certificado_energetico,consumoce_ano,letrace,emisiones_co2
0,Disponible,"≥ 3,51",NaN,D
1,No indicado,desconocido,NaN,No indicado
2,No indicado,desconocido,NaN,No indicado
3,No indicado,desconocido,NaN,No indicado
4,Disponible,desconocido,NaN,No indicado
...,...,...,...,...
9298,NaN,NaN,NaN,34.00
9317,NaN,NaN,NaN,45.00
9318,NaN,NaN,NaN,86.00
9431,NaN,NaN,B,NaN


In [4726]:
df17["emisiones_co2"].value_counts(dropna=False)

emisiones_co2
 NaN                    6800
 No indicado            2027
 Inclasificable          289
 E                       122
 D                        97
                        ... 
 45.7 kg CO₂/m² añoE       1
 26.3 kg CO₂/m² añoD       1
 321 kg CO₂/m² añoE        1
 13 kg CO₂/m² añoC         1
86.00                      1
Name: count, Length: 350, dtype: int64

In [4727]:
df17["consumoce_ano"].unique()

array(['≥ 3,51', 'desconocido', '90', '258.5', '129', '284', '207', '145',
       '170', '77', '138', '120', '231.1', '224', '191', '50', '194.2',
       '216', '121', '85', '123', '399', '999', '41.2', '294', '300',
       '173', '186', '193', '400', '170.3', '172.2', '444', '176',
       '209.7', '24', '74', '164', '163', '222', '2', '328', '242', '78',
       '89', '87', '382', '342', '441', '767', '111.1', '312', '199',
       '407', '336.3', '132', '266', '55', '248', '38', '463', '188',
       '126', '414', '155', '460', '301', '44', '354', '285', '178',
       '272', '260', '324', '221', '200.8', '35', '40', '47', '158',
       '262', '130', '114.1', '212', '192', '68.5', '214', '117', '186.9',
       '245', '299', '204', '137.9', '166', '55.5', '160', '255', '189',
       '195', '263', '244', '135', '180', '110', '321', '225', '313',
       '200', '149', nan], dtype=object)

In [4728]:
# Asignar letrace a letra_ce solo cuando letra_ce es nulo y letrace no es nulo
df17.loc[df17['letra_ce'].isnull() & df17['letrace'].notnull(), 'letra_ce'] = df17['letrace']
df17=df17.drop(columns='letrace')

In [4729]:
#se encuentra un piso que han modificado los valores en la web
# debe tener https://www.indomio.es/anuncios/97003125/, Consumo de energia:57.9 kWh/m² año, letrace:B, e Emisiones de CO₂:11.08 kg CO₂/m² año B
# Asignar valores específicos si la URL coincide
for index, value in df17["url"].items():
    if value == "https://www.indomio.es/anuncios/97003125/":
        df17.at[index, "consumoce_ano"] = 57.9
        df17.at[index, "letra_ce"] = "B"
        df17.at[index, "emisiones_co2"] = "11.08 kg CO₂/m² año B"


In [4730]:
df17["letra_ce"].value_counts(dropna=False)

letra_ce
NaN    8280
E       572
e       332
D       248
F       174
G       134
d       126
C        96
A        48
f        46
c        46
B        40
g        28
a        18
b        15
Name: count, dtype: int64

#

CONSUMO Y LETRA
A: Menos de 50 kWh/m²/año
B: 51-90 kWh/m²/año
C: 91-150 kWh/m²/año
D: 151-230 kWh/m²/año
E: 231-330 kWh/m²/año
F: 331-450 kWh/m²/año
G: Más de 450 kWh/m²/año

En el caso de las emisiones:
A: Menos de 10 kg CO2/m²/año
B: 10-20 kg CO2/m²/año
C: 20-35 kg CO2/m²/año
D: 35-55 kg CO2/m²/año
E: 55-75 kg CO2/m²/año
F: 75-100 kg CO2/m²/año
G: Más de 100 kg CO2/m²/año

In [4731]:
#Revisar que filas no tiene descripcion_larga , deberian tenerlas todas
df17.query("descripcion_larga.isnull()")['web'].unique()

array(['Indomio.com', 'pisos.com', 'redpiso', 'pisosmadrid'], dtype=object)

In [4732]:
df17['zonas_verdes'].value_counts(dropna=False)

zonas_verdes
NaN     10072
1.00      131
Name: count, dtype: int64

In [4733]:
porcentaje_nulos(df17,['zonas_verdes','nombre_oficina'])

{'zonas_verdes': 98.7160639027737, 'nombre_oficina': 85.69048319121826}

In [4734]:
df17.loc[df17['consumoce_ano'].isnull(), 'consumoce_ano'] = df17.loc[df17['consumoce_ano'].isnull(), 'consumo_energia'].apply(lambda x: extraer_numero(x) if isinstance(x, str) else None)

In [4735]:
columns_to_replace = ['consumoce_ano','letra_ce', 'emisiones_co2']
df17[columns_to_replace] = df17[columns_to_replace].replace(['desconocido', 'No indicado', 'Inclasificable','Pendiente','Exento','Cal energetica','Cal energetica Exento '], np.nan)

In [4736]:
df17["letra_ce"].value_counts(dropna=False)

letra_ce
NaN    8280
E       572
e       332
D       248
F       174
G       134
d       126
C        96
A        48
f        46
c        46
B        40
g        28
a        18
b        15
Name: count, dtype: int64

In [4737]:
df17["emisiones_co2"].value_counts(dropna=False)

emisiones_co2
 NaN                    9144
 E                       122
 D                        97
 F                        77
 C                        46
                        ... 
 45.7 kg CO₂/m² añoE       1
 26.3 kg CO₂/m² añoD       1
 321 kg CO₂/m² añoE        1
 13 kg CO₂/m² añoC         1
86.00                      1
Name: count, Length: 346, dtype: int64

In [4738]:
df17['emisiones_co2'] = df17['emisiones_co2'].str.replace('kg CO₂/m² año', '')


In [4739]:
# Reemplazar 'NaN' con valores nulos de Pandas
df17['emisiones_co2'] = df17['emisiones_co2'].replace('NaN', np.nan)

# Extraer letra_emisiones y emisiones_ano
df17[['emisiones_ano', 'letra_emisiones']] = df17['emisiones_co2'].str.extract(r'(?:(\d+\.\d+|\d+)\s)?([A-Z])')

# Convertir emisiones_ano a float
df17['emisiones_ano'] = df17['emisiones_ano'].astype(float)

In [4740]:
df17['letra_emisiones'].value_counts(dropna=False)

letra_emisiones
NaN    9405
E       384
D       154
F       105
C        60
G        58
A        19
B        18
Name: count, dtype: int64

In [4741]:
df17['emisiones_ano'].value_counts(dropna=False)

emisiones_ano
NaN      9816
44.00      11
42.00       9
56.00       8
36.00       8
         ... 
56.90       1
49.80       1
58.10       1
18.00       1
86.00       1
Name: count, Length: 180, dtype: int64

In [4742]:
df17.query("consumoce_ano.notnull() and letra_ce.isnull()")[['url','consumoce_ano','letra_ce','emisiones_co2']]

,url,consumoce_ano,letra_ce,emisiones_co2
7872,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126007,111,NaN,18.0
7873,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126427,145,NaN,24.0
7876,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126442,104,NaN,17.0
7882,https://www.redpiso.es/inmueble/piso-en-venta-en-buenavista-carabanchel-madrid-madrid-RP442024126423,88,NaN,14.0
7883,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-alejandro-sanchez-opanel-carabanchel-madrid-madrid-RP512023115395,600,NaN,149.0
...,...,...,...,...
9287,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-fresnedillas-fuentelarreina-fuencarral-el-pardo-madrid-madrid-RP722023118677,198,NaN,NaN
9292,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-general-pardinas-58-lista-salamanca-madrid-madrid-RP1432024124685,326,NaN,NaN
9298,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-orense-cuatro-caminos-tetuan-madrid-madrid-RP2332023114339,138,NaN,NaN
9317,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-casado-del-alisal-jeronimos-retiro-madrid-madrid-RP1792023115851,180,NaN,NaN


In [4743]:
df17.query("consumo_energia.notnull() or letra_ce.notnull() or emisiones_co2.notnull()")[['url','consumoce_ano','letra_ce','emisiones_co2','letra_emisiones','emisiones_ano']]

,url,consumoce_ano,letra_ce,emisiones_co2,letra_emisiones,emisiones_ano
0,https://www.indomio.es/anuncios/75404850/,"≥ 3,51",F,D,D,NaN
4,https://www.indomio.es/anuncios/82164739/,NaN,E,NaN,NaN,NaN
11,https://www.indomio.es/anuncios/83875795/,"≥ 3,51",F,NaN,NaN,NaN
12,https://www.indomio.es/anuncios/84201619/,"≥ 3,51",G,NaN,NaN,NaN
13,https://www.indomio.es/anuncios/84201619/,"≥ 3,51",G,NaN,NaN,NaN
...,...,...,...,...,...,...
9298,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-orense-cuatro-caminos-tetuan-madrid-madrid-RP2332023114339,138,NaN,NaN,NaN,NaN
9317,https://www.redpiso.es/inmueble/piso-en-venta-en-calle-casado-del-alisal-jeronimos-retiro-madrid-madrid-RP1792023115851,180,NaN,NaN,NaN,NaN
9318,https://www.redpiso.es/inmueble/piso-en-venta-en-lista-salamanca-madrid-madrid-RP1642024124993,417,NaN,NaN,NaN,NaN
9431,https://www.pisosmadrid.com.es/propiedad/3906133-piso-en-venta-en-madrid,None,B,NaN,NaN,NaN


In [4744]:
pd.set_option('display.max_rows', None)

In [4745]:
#eliminar las columnas consumo_energia,cal_energetica

In [4746]:
df17 = df17.drop(columns=['consumo_energia'])

In [4747]:


# Palabras clave para clasificar
central_keywords = ["central", "comunitaria", "comunitario"]
individual_keywords = ["individual", "particular", "chimenea", "bomba", "termo", "climatizacion"]
electric_keywords = ["electrica", "bomba", "climatización", 'termo electrico','climatizacion']
gas_keywords = ["gas", "propano", "g n", "g/n"]  # Asegurarse de que 'g n' y 'g/n' estén bien escritos
gasoil_keywords = ["gasóil","gasoil",'gasoil',"Gasoil"]  # Añadir "gasoil" aquí
renovables_keywords = ["aerotermia", "geotermia", "chimenea"]
suelo_radiante_keywords=["suelo_radiante", "suelo"]
#radiadores_keywords=["radiadores"]
aire_keywords=["bomba","climatización",'climatizacion','conducto']
desconocido_keywords=['chimenea', 'termo electrico']
#lo mas extendido son los radiadores por eso despues de pasar , suelo radiante y aire pongo a todos radiadores salvo a chimenea y termo electrico que no lo se
radiadores_keywords=['radiadores','Gas natural', 'G/n', 'Gas particular', 'Central, alimentación eléctrica', 'Agua caliente y calefacción central con contador i', 'Comunitario', 'Individual con gas natural', 'Individual, de gasóil', 'Comunitaria', 'Calefaccion Propano', 'Calefaccion Gas Natural', 'Central, por aire, de gas', 'Gas natural y biomasa', 'Centralizada', 'Eléctrica', 'Calefaccion Individual', 'Calefaccion individual','Contadores individuales', '1.0', 'Individual, alimentación eléctrica', 'Aerotermia', 'Individual - gas natural', 'Individual', 'Calefaccion Aerotermia', 'Gasoil', 'Central individualizada', 'Individual, por aire, alimentación eléctrica', 'Geotermia y aerotermia', 'Calefaccion comunitaria', 'Calefaccion Central', 'Central, de gas', 'Calefaccion Electrica', 'Calefacción central individualizada', 'Central, de gasóil', 'Individual, de gas', 'Central']

# Inicializar conjuntos para valores únicos
central = set()
individual = set()
electrica = set()
gas = set()
gsoil = set()
renovables = set()
suelo_radiante = set()
radiadores = set()
aire = set()
sin_clasificar = set()
sin_clasificar_energia = set()
sin_clasificar_medio = set()
radiadores=set()
suelo_radiante=set()


# Clasificar categorías
for categoria in df17["calefaccion"]:
    categoria_normalized = normalize_text(str(categoria))  # Normalizar texto



    # Clasificación por tipo (central o individual)
    if any(keyword in categoria_normalized for keyword in central_keywords):
        central.add(categoria)
    elif any(keyword in categoria_normalized for keyword in individual_keywords):
        individual.add(categoria)
    else:
        sin_clasificar.add(categoria)

    # Clasificación por energía (eléctrica, gas, gasoil)
    if any(keyword in categoria_normalized for keyword in electric_keywords):
        electrica.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gasoil_keywords):
        gsoil.add(categoria)
    elif any(keyword in categoria_normalized for keyword in gas_keywords):
        gas.add(categoria)


    elif any(keyword in categoria_normalized for keyword in renovables_keywords):
        renovables.add(categoria)
    else:
        sin_clasificar_energia.add(categoria)

    # Clasificación por medio (suelo radiante, radiadores, aire)


    if any(keyword in categoria_normalized for keyword in aire_keywords):
        aire.add(categoria)
    elif any(keyword in categoria_normalized for keyword in suelo_radiante_keywords):
        suelo_radiante.add(categoria)
    elif any(keyword in categoria_normalized for keyword in desconocido_keywords):
        sin_clasificar_medio.add(categoria)
  #  else: any(keyword in categoria_normalized for keyword in radiadores_keywords):
    else:
        radiadores.add(categoria)


# Convertir los conjuntos a listas
central = list(central)
individual = list(individual)
sin_clasificar = list(sin_clasificar)
electrica = list(electrica)
gas = list(gas)
gasoil = list(gsoil)
renovables = list(renovables)
sin_clasificar_energia = list(sin_clasificar_energia)
aire=list(aire)
radiadores=list(radiadores)
suelo_radiante=list(suelo_radiante)
sin_clasificar_medio=list(sin_clasificar_medio)
# Imprimir resultados
print("Central:", central)
print("Individual:", individual)
print("No clasificado:", sin_clasificar)
print("Electrica:", electrica)
print("Gas:", gas)
print("Gasoil:", gasoil)
print("Renovables:", renovables)
print("No clasificado en energía:", sin_clasificar_energia)
print("Suelo Radiante:", suelo_radiante)
print("Radiadores:", radiadores)
print("Aire:", aire)
print("No clasificado en medio:", sin_clasificar_medio)


Central: ['Central, por aire, de gas', 'Calefaccion comunitaria', 'Central, suelo radiante, de gas', 'Central, de gas', 'Agua caliente y calefacción central con contador i', 'Central', 'Comunitario', 'Centralizada', 'Central, alimentación eléctrica', 'Central, por radiadores, de gasóil', 'Central, suelo radiante', 'Central, por radiadores, de gas', 'central por gas', 'Comunitaria', 'Central, de gasóil', 'Central, por radiadores', 'Central individualizada', 'central', 'Calefacción central individualizada']
Individual: ['Individual', 'individual gas natural', 'Individual, de gas', 'Termo electrico', 'Individual con gas natural', 'Bomba de calor', 'Climatizacion', 'Individual, suelo radiante, de gas', 'Individual, por radiadores', 'climatizacion', 'Bomba frio/calor', 'Bomba frio-calor', 'Por climatizacion', 'Contadores individuales', 'Individual, por radiadores, de gas', 'Individual, por radiadores, de gasóil', 'Individual, por radiadores, alimentación eléctrica', 'Individual, suelo radia

In [4748]:
# Listas proporcionadas
listas = {
    'Central': ['Calefaccion comunitaria', 'Agua caliente y calefacción central con contador i', 'Central', 'Calefaccion Central', 'Calefacción central individualizada', 'Central, por radiadores, de gas', 'Comunitario', 'Central, por aire, de gas', 'Central, por radiadores', 'Comunitaria', 'Central individualizada', 'Central, de gas', 'Central, de gasóil', 'Centralizada', 'Central, por radiadores, de gasóil', 'Central, suelo radiante, de gas', 'Central, suelo radiante', 'Central, alimentación eléctrica'],
    'Individual': ['Gas particular', 'Climatizacion', 'Individual, suelo radiante', 'Calefaccion individual', 'Contadores individuales', 'climatizacion', 'Individual, por radiadores', 'Individual, por radiadores, de gasóil', 'Chimenea', 'Individual, alimentación eléctrica', 'Bomba frio-calor', 'Termo electrico', 'Individual, por aire, por bomba de calor', 'Individual, por radiadores, de gas', 'Individual con gas natural', 'Individual, suelo radiante, de gas', 'Bomba de calor', 'Individual, por aire, alimentación eléctrica', 'Bomba frio/calor', 'Individual - gas natural', 'Bomba frio calor', 'Individual, de gas', 'Calefaccion Individual', 'Individual, por radiadores, alimentación eléctrica', 'Por climatizacion', 'Individual', 'Individual, de gasóil'],
    'No clasificado': ['Calefaccion Propano', 'Calefaccion Electrica', 'Eléctrica', 'Suelo radiante agua', 'Conducto', 'Gasoil', 'Calefaccion Gas Natural', 'G/n', 'Geotermia y aerotermia', 'Calefaccion Aerotermia', '1.0', 'Calefaccion Suelo Radiante', 'Aerotermia', np.nan, 'Gas natural', 'Gas natural y biomasa', 'Suelo radiante'],
    'Electrica': ['Individual, por aire, por bomba de calor', 'Individual, por aire, alimentación eléctrica', 'Bomba frio calor', 'Calefaccion Electrica', 'Individual, por radiadores, alimentación eléctrica', 'Climatizacion', 'Eléctrica', 'Bomba frio/calor', 'Por climatizacion', 'climatizacion', 'Individual, alimentación eléctrica', 'Bomba de calor', 'Bomba frio-calor', 'Termo electrico', 'Central, alimentación eléctrica'],
    'Gas': ['Calefaccion Propano', 'Gas particular', 'Individual, por radiadores, de gas', 'Individual, de gas', 'Central, por radiadores, de gas', 'Individual con gas natural', 'Individual, suelo radiante, de gas', 'Central, por aire, de gas', 'Calefaccion Gas Natural', 'G/n', 'Individual - gas natural', 'Central, de gas', 'Central, suelo radiante, de gas', 'Gas natural', 'Gas natural y biomasa'],
    'Gasoil': ['Gasoil', 'Individual, por radiadores, de gasóil', 'Central, de gasóil', 'Central, por radiadores, de gasóil', 'Individual, de gasóil'],
    'Renovables': ['Calefaccion Aerotermia', 'Chimenea', 'Aerotermia', 'Geotermia y aerotermia'],
    'No clasificado en energía': ['Comunitario', 'Individual, suelo radiante', 'Calefaccion individual', 'Contadores individuales', 'Individual, por radiadores', 'Centralizada', 'Suelo radiante', 'Calefaccion Central', 'Calefacción central individualizada', 'Conducto', 'Central, suelo radiante', 'Agua caliente y calefacción central con contador i', 'Comunitaria', 'Central individualizada', 'Calefaccion Suelo Radiante', np.nan, 'Central, por radiadores', 'Calefaccion comunitaria', 'Central', 'Calefaccion Individual', 'Suelo radiante agua', '1.0', 'Individual'],
    'Suelo Radiante': ['Individual, suelo radiante, de gas', 'Suelo radiante agua', 'Individual, suelo radiante', 'Calefaccion Suelo Radiante', 'Central, suelo radiante, de gas', 'Central, suelo radiante', 'Suelo radiante'],
    'Radiadores': ['Gas particular', 'Calefaccion Electrica', 'Central, por radiadores, de gas', 'Comunitario', 'Central, por aire, de gas', 'Eléctrica', 'Gasoil', 'Geotermia y aerotermia', 'Calefaccion individual', 'Contadores individuales', 'Calefaccion Aerotermia', 'Individual, por radiadores', 'Individual, por radiadores, de gasóil', 'Centralizada', 'Individual, alimentación eléctrica', 'Calefaccion Central', 'Individual, por radiadores, de gas', 'Calefacción central individualizada', 'Individual con gas natural', 'G/n', 'Central, de gas', 'Central, de gasóil', 'Central, por radiadores, de gasóil', 'Agua caliente y calefacción central con contador i', 'Calefaccion Propano', 'Individual, por aire, alimentación eléctrica', 'Calefaccion Gas Natural', 'Comunitaria', 'Individual - gas natural', 'Central individualizada', np.nan, 'Central, por radiadores', 'Central, alimentación eléctrica', 'Calefaccion comunitaria', 'Central', 'Individual, de gas', 'Calefaccion Individual', 'Individual, por radiadores, alimentación eléctrica', '1.0', 'Individual', 'Aerotermia', 'Gas natural', 'Gas natural y biomasa', 'Individual, de gasóil'],
    'Aire': ['Individual, por aire, por bomba de calor', 'Bomba frio calor', 'Climatizacion', 'Conducto', 'Bomba frio/calor', 'Por climatizacion', 'climatizacion', 'Bomba de calor', 'Bomba frio-calor'],
    'No clasificado en medio': ['Termo electrico', 'Chimenea']
}

In [4749]:
# Definir las listas específicas para cada columna nueva
listas_tipo_1 = {k: listas[k] for k in ['Central', 'Individual']}
listas_tipo_2 = {k: listas[k] for k in ['Electrica', 'Gas', 'Gasoil','Renovables','No clasificado en energía']}
listas_tipo_3 = {k: listas[k] for k in ['Suelo Radiante', 'Radiadores', 'Aire', 'No clasificado en medio']}

# Aplicar la función a la columna calefaccion y crear tres nuevas columnas
df17['tipo_calefaccion'] = df17['calefaccion'].apply(lambda x: clasificar_calefaccion(x, listas_tipo_1))
df17['energia_calefaccion'] = df17['calefaccion'].apply(lambda x: clasificar_calefaccion(x, listas_tipo_2))
df17['instalacion_calefaccion'] = df17['calefaccion'].apply(lambda x: clasificar_calefaccion(x, listas_tipo_3))



In [4750]:
df17["amueblado"].value_counts(dropna=False)

amueblado
 NaN                       8955
 1.0                        575
 Sí                         274
1.00                        159
 No                         158
 Sólo cocina amueblada       69
 Parcialmente amueblado      13
Name: count, dtype: int64

In [4751]:
df17.isna().mean()*100

title                                        0.00
url                                          0.00
precio_anterior                             99.46
descuento                                   99.46
EUR/m2                                      23.02
m2_constr                                    0.05
distrito                                     0.07
cod_distrito                                 0.07
barrio                                       0.67
cod_barrio                                   0.67
zona                                        15.54
calle                                       47.41
consumoce_ano                               95.22
letra_ce                                    81.15
emisiones_co2                               91.57
tipologia                                    0.00
estado                                      45.23
amueblado                                   87.77
planta                                      45.57
dormitorios                                  2.22


In [4752]:
# Chequeo de valores en columna
df17["alarma"].value_counts()

alarma
0                                                3100
1                                                  52
1                                                  42
Portero físico                                     12
vigilancia 24h, con cámaras de seguridad            5
Conserje                                            4
con cámaras de seguridad                            3
Portero fisico                                      2
24 horas                                            2
Servicio de portería                                2
24h                                                 2
Conserje 24 horas                                   1
vigilancia 24h                                      1
Circuito de video vigilancia en zonas comunes       1
Cámaras de vigilancia en portal                     1
Camaras                                             1
Portería. Portero físico                            1
Portero fisico de dia.                              1
Puerta de seguridad  

In [4753]:
df17["puerta_seguridad"].value_counts()

puerta_seguridad
1.00    18
Name: count, dtype: int64

In [4754]:
df17["puerta_blindada"].value_counts()

puerta_blindada
0                                                     3050
1                                                       92
Acorazada                                                2
Tiene 3 puertas de acceso                                1
Muy cerca centro comercial las rosas, metro y bus.       1
Puerta acorazada                                         1
Bombillo seguridad multilock                             1
Puerta a la calle                                        1
Puerta de entrada blindada                               1
Name: count, dtype: int64

In [4755]:
df17["zonas_verdes"].value_counts()

zonas_verdes
1.00    131
Name: count, dtype: int64

In [4756]:
df17["codigo_Postal"].unique()

array([28001., 28042., 28045., 28016., 28020., 28002., 28015., 28021.,
       28024., 28006., 28004., 28035., 28031., 28053., 28013., 28010.,
       28028., 28046., 28014., 28037., 28027., 28055., 28012., 28008.,
       28011., 28019., 28003., 28009., 28007., 28039., 28033., 28032.,
       28044., 28041., 28043., 28017., 28018., 28029., 28023., 28038.,
       28034., 28047., 28026., 28048., 28025., 28040., 28030., 28005.,
          nan, 28049.])

In [4757]:
df17.loc[df17['url'].str.contains('calle-narvaez'), ['cod_barrio', 'barrio']] = [34, 'Ibiza']
df17.loc[df17['url'].str.contains('calle-claudio-coello'), ['cod_barrio', 'barrio']] = [41, 'Recoletos']
df17.loc[df17['url'].str.contains('calle-maria-de-molina'), ['cod_barrio', 'barrio']] = [51, 'El Viso']
df17.loc[df17['url'].str.contains('calle-eloy-gonzalo'), ['cod_barrio', 'barrio']] = [73, 'Trafalgar']

df17['titular_anuncio'] = df17['descripcion_larga'].fillna('')
df17.loc[df17['titular_anuncio'].str.contains('Ibiza'), ['cod_barrio', 'barrio']] = [34, 'Ibiza']


# Reemplazar valores NaN en 'descripcion_larga' con una cadena vacía
df17['descripcion_larga'] = df17['descripcion_larga'].fillna('')
df17.loc[df17['descripcion_larga'].str.contains('Calle Mariano Benlliure'), ['cod_barrio', 'barrio','cod_distrito','distrito']] = [74, 'Almagro',7,'Chamberí']
df17.loc[df17['descripcion_larga'].str.contains('Calle Moquetas 11'), ['cod_barrio', 'barrio','cod_distrito','distrito','estado']] = [74, 'Almagro',7,'Chamberí','Obra nueva']
df17.loc[df17['descripcion_larga'].str.contains('calle de Castello'), ['cod_barrio', 'barrio']] = [74, 'Almagro']
df17.loc[df17['descripcion_larga'].str.contains('IBIZA'), ['cod_barrio', 'barrio']] = [34, 'Ibiza']

In [4758]:
# Convertir las columnas a enteros, ignorando los errores
df17['cod_distrito'] = pd.to_numeric(df17['cod_distrito']).astype('Int64')
df17['cod_barrio'] = pd.to_numeric(df17['cod_barrio']).astype('Int64')

In [4759]:
# Concatenar cod_barrio y cod_distrito si cod_barrio < 11
df17.loc[df17['cod_barrio'] < 11, 'cod_barrio'] =  df17['cod_distrito'].astype(str)+df17['cod_barrio'].astype(str)


In [4760]:
df17["m2_utiles"]=df17["m2_utiles"].str.replace("Metros","")
df17["m2_utiles"]=df17["m2_utiles"].str.replace("m²","")

In [4761]:
df17.drop_duplicates(subset=['url'], keep='first', inplace=True)


In [4762]:
df17['url'].nunique() == len(df17)

True

In [4763]:
df17.drop_duplicates(subset=['url'], keep='first', inplace=True)


In [4764]:
df17.precio.describe()

count      10113.00
mean      887858.30
std      1160881.31
min        25000.00
25%       239000.00
50%       477500.00
75%      1080000.00
max     19950000.00
Name: precio, dtype: float64

In [4765]:
# Replace values in the 'portero' column
df17['portero'] = np.where(df17['portero'] == 'Portero media jornada', 0.5, df17['portero'])
df17['portero'] = np.where(df17['portero'] == 'Portero todo el día', 1, df17['portero'])
df17['portero'] = np.where(df17['portero'] == '1', 1, df17['portero'])

In [4766]:
# Eliminar el símbolo '%' y convertir a tipo numérico
df17['descuento'] = df17['descuento'].str.replace('%', '')
df17['descuento'] = pd.to_numeric(df17['descuento'].str.replace(',', '.'))
#df17["alarma"] = df17["alarma"].fillna(df17["sistema_alarma"])
#puerta de seguridad y puerta_blindada en una columna
df17["puerta_seguridad"] = df17["puerta_seguridad"].fillna(df17["puerta_blindada"])
df17=df17.drop(columns="puerta_blindada")

In [4767]:
df17["letra_ce"].value_counts(dropna=False)

letra_ce
NaN    8220
E       563
e       332
D       238
F       170
G       131
d       126
C        93
A        47
f        46
c        46
B        40
g        28
a        18
b        15
Name: count, dtype: int64

In [4768]:
# Aplicar la función a la columna consumo_ce solo cuando letrace es nulo y consumo_ce no es nulo
df17['letra_ce'] = df17.apply(lambda row: clasificar_consumo(row['consumoce_ano']) if pd.isnull(row['letra_ce']) and not pd.isnull(row['consumoce_ano']) else row['letra_ce'], axis=1)

In [4769]:
df17= df17[df17['tipologia'] != 'Negocio']
df17['tipologia'] = df17['tipologia'].replace('Venta', 'Piso')
# Actualizar los valores de 'distrito' y 'barrio' cuando 'cod_barrio' es 195
df17.loc[df17['cod_barrio'] == 195.00, ['distrito', 'barrio']] = ['Vicalvaro', 'Los Berrocales']

In [4770]:
df_final=df17

In [4771]:
df_final.shape

c:\Users\extas\anaconda3\envs\Nuclio\Lib\site-packages\IPython\core\displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


(10109, 93)

In [4772]:
df_final.columns

Index(['title', 'url', 'precio_anterior', 'descuento', 'EUR/m2', 'm2_constr',
       'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona', 'calle',
       'consumoce_ano', 'letra_ce', 'emisiones_co2', 'tipologia', 'estado',
       'amueblado', 'planta', 'dormitorios', 'banos', 'balcon', 'terraza',
       'garaje', 'trastero', 'ascensor', 'calefaccion', 'antiguedad',
       'num_plantas', 'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'nuda_propiedad', 'tipo_inmueble',
       'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpinteria_exterior_vidrio/metal', 'chimenea', 'cocina',
       'fibra_optica', 'exterior', 'interi

In [4773]:
# Aplicar la lógica: valores no nulos y diferentes de 0.00 se convierten en 1
df_final['garaje'] = df_final['garaje'].apply(lambda x: 1 if pd.notna(x) and x!=0.00 else x)

In [4774]:
df_final.loc[df_final['num_garajes'] == 2, 'garaje'] = 1

In [4775]:
df_final['garaje'].unique()

array([ 1., nan,  0.])

In [4776]:
df_final["interior_y_exterior"].value_counts(dropna=False)

interior_y_exterior
NaN     7057
0.00    3044
1.00       8
Name: count, dtype: int64

In [4777]:
# Asignar 1 a 'interior' y 'exterior' cuando 'interior_y_exterior' es igual a 1.00
df_final.loc[df_final['interior_y_exterior'] == 1.00, ['interior', 'exterior']] = 1


In [4778]:
#vamos a agrupar algunas categorias que están repetidas con acentos y sin acentos

df_final["tipologia"]=df_final["tipologia"].replace({'Ático': 'Atico', 'Dúplex': 'Duplex'})
df_final["distrito"]=df_final["distrito"].replace({'Puente de Vallecas': 'Puente De Vallecas'})


#Convertir los valores de 'letra_ce' a mayúsculas
df_final['letra_ce'] = df_final['letra_ce'].str.upper()
df_final['letra_emisiones'] = df_final['letra_ce'].str.upper()
df_final['estado'] = df_final['estado'].str.strip().str.title()



In [4779]:
# Rellenar los valores nulos de 'garajes' con los valores de 'num_garajes'
df_final['garaje'] = df_final['garaje'].fillna(df_final['num_garajes'])

In [4780]:
#Elimino las filas con cod_barrio nulo
# Eliminar filas donde 'cod_barrio' es nulo
df_final = df_final.dropna(subset=['cod_barrio'])
#eLIMINO LAS FILAS con M2_CONSTR nulo
df_final=df_final.dropna(subset=['m2_constr'])

In [4781]:
df_final.reset_index(drop=True, inplace=True)

In [4782]:
#columna 45 acceso_discapacidad, eliminar los valores
val=['Exteriores' 'Ver todas las características' 'Otros']
df_final['acceso_discapacidad'] = df_final['acceso_discapacidad'].replace(val, np.nan)
# Replace '.0' and similar patterns with empty string using regex
df_final['acceso_discapacidad'] = df_final['acceso_discapacidad'].astype(str).str.replace('.0', '', regex=True)
df_final['acceso_discapacidad'] = pd.to_numeric(df_final['acceso_discapacidad'], errors='coerce').astype('Int64') # Convert to numeric, handle errors, then to Int64

In [4783]:
cambios = {'3+': '5',  '.0': '','0.0':np.nan,'nan':np.nan}
# Reemplazar los valores '3+' por 5 en la columna 'banos'
df_final['banos'] = df_final['banos'].astype(str).replace(cambios).str.strip()


In [4784]:
#56 puerta_seguridad
cambios = {'Puerta acorazada': '1',  'Bombillo seguridad multilock': '1', 'Acorazada': '1','Puerta de entrada blindada':'1','Puerta a la calle':'1','1.0':'1'}
cambiosnan={'Tiene 3 puertas de acceso','Muy cerca centro comercial las rosas, metro y bus.'}
df_final['puerta_seguridad'] = df_final['puerta_seguridad'].astype(str).replace(cambios).str.strip()
df_final['puerta_seguridad'] = df_final['puerta_seguridad'].replace(cambiosnan,np.nan)
df_final['puerta_seguridad'] = pd.to_numeric(df_final['puerta_seguridad'], errors='coerce')
# Change the astype to pd.Int64Dtype() to handle NaN
df_final['puerta_seguridad'] = df_final['puerta_seguridad'].astype(pd.Int64Dtype())

In [4785]:
cambios = {'Más de 2': '4', 'Mas de 5': '7', '.0': ''}
df_final['armarios'] = df_final['armarios'].replace(cambios).str.strip()


# creacion dataframe EDA

In [4786]:
df_final.columns.tolist()

['title',
 'url',
 'precio_anterior',
 'descuento',
 'EUR/m2',
 'm2_constr',
 'distrito',
 'cod_distrito',
 'barrio',
 'cod_barrio',
 'zona',
 'calle',
 'consumoce_ano',
 'letra_ce',
 'emisiones_co2',
 'tipologia',
 'estado',
 'amueblado',
 'planta',
 'dormitorios',
 'banos',
 'balcon',
 'terraza',
 'garaje',
 'trastero',
 'ascensor',
 'calefaccion',
 'antiguedad',
 'num_plantas',
 'aire_acondicionado',
 'jardin',
 'inmueble_ingresos',
 'alquiler_opcion_a_compra',
 'nuda_propiedad',
 'tipo_inmueble',
 'tiene_armario',
 'cancha_tenis',
 'carpinteria_exterior_doble_vidrio/pvc',
 'carpinteria_exterior_doble_vidrio/madera',
 'carpinteria_exterior_doble_vidrio/metal',
 'carpinteria_exterior_triple_vidrio/madera',
 'carpinteria_exterior_triple_vidrio/metal',
 'carpinteria_exterior_vidrio/pvc',
 'carpinteria_exterior_vidrio/madera',
 'carpinteria_exterior_vidrio/metal',
 'chimenea',
 'cocina',
 'fibra_optica',
 'exterior',
 'interior',
 'interior_y_exterior',
 'piscina',
 'porton_electrico',


In [4787]:
df_final["num_pisos"]=df_final["num_pisos"].replace('Pisos:','',regex=True)

In [4788]:
# Eliminar filas donde 'm2_constr' sea igual a 0
df_final = df_final.query("m2_constr != 0")

In [4789]:
df_final.barrio.nunique()

132

In [4790]:
df_final["barrio"]=df_final["barrio"].replace("Niño Jesús",'Niño Jesus',regex=True)

In [4791]:
df_final.barrio.nunique()

131

In [4792]:
# Modificar los valores según la condición
df_final.loc[(df_final['tiene_armario'] == 1) & (df_final['armarios'] == 0), 'armarios'] = 1

In [4793]:
# Cargar datos adicionales desde una ruta relativa
dfadicionales = pd.read_csv('../02-datos-limpios/Datos_adicionales.csv', sep=";", index_col=False)

In [4794]:
def actualizar_datos(df_principal, df_adicionales,filtro):
    # Filtrar las filas según las condiciones
  
    
    # Iterar sobre las filas que cumplen el filtro
    for index, row in df_principal[filtro].iterrows():
        cod_barrio = row['cod_barrio']
        
        # Buscar el barrio en el DataFrame adicional
        match = df_adicionales[df_adicionales['cod_barrio'] == cod_barrio]
        if not match.empty:
            # Extraer los valores necesarios
            cod_distrito = match.iloc[0]['cod_distrito']
            distrito = match.iloc[0]['distrito']
            codigo_postal = match.iloc[0]['codigo_Postal']
            
            # Actualizar el DataFrame principal
            df_principal.at[index, 'cod_distrito'] = cod_distrito
            df_principal.at[index, 'distrito'] = distrito
            df_principal.at[index, 'codigo_Postal'] = codigo_postal
        
    
    return df_principal


In [4795]:
filtro = (df_final['cod_distrito'] == 1) & (df_final['cod_barrio'] > 16)
# Aplicar la función
df_final = actualizar_datos(df_final, dfadicionales,filtro)

In [4796]:
#filtro = df_final.query('cod_distrito == 13 and cod_barrio ==153')
filtro = (df_final['cod_distrito'] == 13) & (df_final['cod_barrio'] ==153)
df_final = actualizar_datos(df_final, dfadicionales,filtro)

In [4797]:
#filtro = df_final.query('cod_distrito == 12 and cod_barrio <121')
filtro = (df_final['cod_distrito'] == 12) & (df_final['cod_barrio'] <121)
df_final = actualizar_datos(df_final, dfadicionales,filtro)

In [4798]:
filtro = (df_final['cod_distrito'] == 11) & (df_final['cod_barrio'] <120)
#filtro = df_final.query('cod_distrito == 11 and cod_barrio <120')
df_final = actualizar_datos(df_final, dfadicionales,filtro)

In [4799]:
filtro = (df_final['cod_distrito'] == 10) & (df_final['cod_barrio'] ==21)
#filtro = df_final.query('cod_distrito == 10 and cod_barrio==21')
df_final = actualizar_datos(df_final, dfadicionales,filtro)

In [4800]:
filtro = (df_final['cod_distrito'] == 7) & (df_final['cod_barrio'] <70)
#filtro = df_final.query('cod_distrito == 7 and cod_barrio<70')
df_final = actualizar_datos(df_final, dfadicionales,filtro)

In [4801]:
filtro = (df_final['cod_distrito'] == 17) & (df_final['cod_barrio'] ==117)
#filtro = df_final.query('cod_distrito == 7 and cod_barrio<70')
df_final = actualizar_datos(df_final, dfadicionales,filtro)

In [4802]:
filtro = (df_final['cod_distrito'] == 6) & (df_final['cod_barrio'] ==56)
#filtro = df_final.query('cod_distrito == 6 and cod_barrio==56')


    # Filtrar las filas según las condiciones
  
    
    # Iterar sobre las filas que cumplen el filtro
for index, row in df_final[filtro].iterrows():
        
    # Extraer los valores necesarios
    distrito='Tetuán'
    cod_distrito = 6
    cod_barrio=65
    barrio='Valdeacederas' 
    cp=28029
    
            
            # Actualizar el DataFrame principal
    df_final.at[index, 'cod_distrito'] = cod_distrito
    df_final.at[index, 'cod_barrio'] = cod_barrio

    df_final.at[index, 'distrito'] = distrito
    df_final.at[index, 'codigo_Postal'] = cp
        
    
    

In [4803]:
#filtro = df_final.query('cod_distrito == 5 and cod_barrio==34')
filtro = (df_final['cod_distrito'] == 5) & (df_final['cod_barrio'] ==34)


    # Filtrar las filas según las condiciones
  
    
    # Iterar sobre las filas que cumplen el filtro
for index, row in df_final[filtro].iterrows():
        
    # Extraer los valores necesarios
    distrito='Chamartín'
    cod_distrito = 5
    cod_barrio=54
    barrio='Hispanoamérica' 
    cp=28016
    
            
            # Actualizar el DataFrame principal
    df_final.at[index, 'cod_distrito'] = cod_distrito
    df_final.at[index, 'cod_barrio'] = cod_barrio

    df_final.at[index, 'distrito'] = distrito
    df_final.at[index, 'codigo_Postal'] = cp

In [4804]:
#filtro = df_final.query('cod_distrito == 4 and cod_barrio==34')

filtro = (df_final['cod_distrito'] == 4) & (df_final['cod_barrio'] ==34)


    # Filtrar las filas según las condiciones
  
    
    # Iterar sobre las filas que cumplen el filtro
for index, row in df_final[filtro].iterrows():
        
    # Extraer los valores necesarios
    distrito='Salamanca'
    cod_distrito = 4
    cod_barrio=42
    barrio='Goya' 
    cp=28001
    
            
            # Actualizar el DataFrame principal
    df_final.at[index, 'cod_distrito'] = cod_distrito
    df_final.at[index, 'cod_barrio'] = cod_barrio

    df_final.at[index, 'distrito'] = distrito
    df_final.at[index, 'codigo_Postal'] = cp

In [4805]:
#18
df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/92616805/',['cod_distrito','distrito','cod_barrio','barrio','cod_postal']]=[18,'Villa De Vallecas',181,'Casco Histórico de Vallecas',28031]
df_final.loc[df_final['url']=='https://www.pisosmadrid.com.es/propiedad/4015338-piso-en-venta-en-vallecas-madrid',['cod_distrito','distrito','cod_barrio','barrio','cod_postal']]=[18,'Villa De Vallecas',181,'Casco Histórico de Vallecas',28031]

df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/96828167/',['cod_distrito','distrito','cod_barrio','barrio','cod_postal']]=[17,'Villaverde',171,'Villaverde Alto',28021]


curls = [
    'https://www.redpiso.es/inmueble/piso-en-venta-en-calle-encomienda-de-palacios-fontarron-moratalaz-madrid-madrid-RP242024131120',
    'https://www.redpiso.es/inmueble/piso-en-venta-en-calle-encomienda-de-palacios-fontarron-moratalaz-madrid-madrid-RP242023119430',
    'https://www.redpiso.es/inmueble/piso-en-venta-en-calle-encomienda-de-palacios-fontarron-moratalaz-madrid-madrid-RP242024124287',
    'https://www.redpiso.es/inmueble/piso-en-venta-en-calle-encomienda-de-palacios-fontarron-moratalaz-madrid-madrid-RP242024127735',
    'https://www.redpiso.es/inmueble/piso-en-venta-en-calle-encomienda-de-palacios-fontarron-moratalaz-madrid-madrid-RP242024128606'
]
df_final.loc[df_final['url'].isin(curls), ['cod_distrito', 'distrito', 'cod_barrio', 'barrio', 'cod_postal']] = [14, 'Moratalaz', 145, 'Fontarrón', 28030]

df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/93066623/',["garaje"]]=['3 privado']
df_final.loc[df_final['url']=='https://www.pisos.com/comprar/chalet-ciudad_universitaria28035-41701368068_101800/',["chimenea","cancha_tenis"]]=[1,1]

df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/96758341/',["garaje","jardin"]]=[0,0]
df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/96760239/',["garaje","jardin"]]=[0,0]
df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/96828189/',["garaje","jardin"]]=[0,0]
df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/95053235/',["garaje","jardin"]]=[0,0]
df_final.loc[df_final['url']=='https://www.redpiso.es/inmueble/piso-en-venta-en-calle-valmojado-aluche-latina-madrid-madrid-RP272024123901',["garaje","piscina",'trastero']]=[0,0,0]
df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/95469291/',["garaje","piscina"]]=[0,0]
df_final.loc[df_final['url']=='https://www.pisos.com/comprar/piso-chamberi_almagro28010-34259046046_996542/',["chimenea"]]=[1]
df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/94254871/',["chimenea"]]=[1]
df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/96905449/',["garaje"]]=['3 privado']

#12


C:\Users\extas\AppData\Local\Temp\ipykernel_21996\4051052091.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['3 privado']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/93066623/',["garaje"]]=['3 privado']


In [4806]:
df_final.query("url=='https://www.indomio.es/anuncios/96905449/'")[['cancha_tenis', 'piscina','terraza','jardin','garaje','trastero','chimenea']]

,cancha_tenis,piscina,terraza,jardin,garaje,trastero,chimenea
2966,0.00,1,1.00,privado,3 privado,0.00,1.00


In [4807]:

#eliminamos esta url , es un edificio

# Eliminar la fila donde la columna 'url' contiene la URL especificada
df_final = df_final[df_final['url'] != 'https://www.indomio.es/anuncios/93990143/']
df_final.loc[df_final['url'].str.contains('https://www.indomio.es/anuncios/93455821/'), ['tipologia', 'terraza', 'estado','garaje']] = ['Loft',1,'Buen estado',1]
df_final.loc[df_final["url"] == "https://www.indomio.es/anuncios/96654135/", "inmueble_ingresos"] = 1

urlsnuevo=['https://www.pisos.com/comprar/apartamento-adelfas28007-45835099732_101800/','https://www.pisos.com/comprar/apartamento-adelfas28007-45869202772_101800/',
'https://www.pisos.com/comprar/apartamento-adelfas28007-45912206979_101800/',
'https://www.pisos.com/comprar/apartamento-adelfas28007-45919489438_101800/',
'https://www.pisos.com/comprar/apartamento-adelfas28007-46667942689_101800/',
'https://www.pisos.com/comprar/apartamento-adelfas28007-46731956156_101800/']
#les asignamos una planta 5, los aticos son mas caros 
# Actualizar las filas correspondientes usando isin
df_final.loc[df_final['url'].isin(urlsnuevo), ['estado', 'trastero', 'tiene_piscina', 'garaje', 'planta']] = ['Obra nueva', 1, 1, 1, 5]

# se elimina la url https://www.pisos.com/comprar/piso-fontarron-47524209698_100600/ por que es un restaurante en venta
url_a_eliminar = 'https://www.pisos.com/comprar/piso-fontarron-47524209698_100600/'
df_final = df_final[df_final['url'] != url_a_eliminar]
#elimino esta fila , tiene datos que no ayudaran al modelo
df_final = df_final[df_final['url'] != 'https://www.indomio.es/anuncios/81008864/']


urls=['https://www.indomio.es/anuncios/96343641/','https://www.indomio.es/anuncios/96535879/']
df_final.loc[df_final['url'].isin(urls), 'num_plantas'] = 3

urls=['https://www.redpiso.es/inmueble/piso-en-venta-en-calle-alvarado-cuatro-caminos-tetuan-madrid-madrid-RP1362024130128',
      'https://www.redpiso.es/inmueble/estudio-en-venta-en-calle-bustamante-delicias-arganzuela-madrid-madrid-RP2102024125647']
df_final.loc[df_final['url'].isin(urls), 'planta'] = 3
# Asignar valores específicos a las columnas 'tipo_inmueble', 'estado' y 'num_garajes' para la URL indicada
df_final.loc[df_final['url'] == 'https://www.pisos.com/comprar/piso-goya28006-39193055272_102100/', ['tipo_inmueble', 'estado', 'garaje']] = ['lujo', 'Obra nueva', '4 privado']
df_final.loc[df_final['url'] == 'https://www.indomio.es/anuncios/95552665/', ['exterior', 'estado', 'garaje']] = [1, 'reformado', '4 privado']
df_final.loc[df_final['url'] == 'https://www.indomio.es/anuncios/96669269/', ['exterior', 'estado', 'grupo_cocina']] = [1, 'reformado', 'abierta, amueblada, equipada']
df_final.loc[df_final['url'] == 'https://www.indomio.es/anuncios/95491161/', ['exterior', 'estado', 'amueblado','grupo_cocina','aire_acondicionado','calefaccion']] = [1, 'reformado', '1','abierta',1,'central']
df_final.loc[df_final['url'] == 'https://www.pisos.com/comprar/duplex-recoletos28001-32597181586_996542/', ['exterior', 'estado', 'antiguedad']] = [1, 'reformado','Más de 50 años']
df_final.loc[df_final['url'] == 'https://www.redpiso.es/inmueble/piso-en-venta-en-paseo-castellana-almagro-chamberi-madrid-madrid-RP1922023117294', ['grupo_cocina','portero', 'estado', 'antiguedad','gastos_comunidad','amueblado','terraza','calefaccion']] = ['equipada',1, 'reformado',1954,1000,'1',1,'central']
df_final.loc[df_final['url'] == 'https://www.indomio.es/anuncios/96662699/', ['cod_barrio','barrio','cod_distrito','distrito','codigo_Postal','grupo_cocina','estado']] = [35,'Jerónimos',3,'Retiro',28014,'equipada','reformado']
df_final.loc[df_final['url'] == 'https://www.indomio.es/anuncios/82878580/', ["garaje",'chimenea',"grupo_cocina"]] = ["1 privado",1,"equipada independiente"]

df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/96736113/',["piscina","orientacion_norte","orientacion_sur","orientacion_este","orientacion_oeste","tipologia","portero","calefaccion","tipo_inmueble"]]=[1,1,1,1,1,"Atico",1,"central","lujo"]
df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/95463509/',["estado"]]=['reformado']

df_final.loc[df_final['url']=='https://www.pisos.com/comprar/piso-recoletos28009-32538606854_149200/',["planta","estado","tipologia","exterior"]]=['bajo','reformado','Chalet',1]
df_final.loc[df_final['url']=='https://www.pisos.com/comprar/chalet-fuencarral_el_pardo_el_pardo28023-46668989227_102100/',["piscina","calefaccion","estado","tipologia","exterior","chimenea","jardin","tiene_jardin","cancha_tenis"]]=['propia','suelo radiante','perfecto estado','Chalet',1,1,"privado",'1',1]
df_final.loc[df_final['url']=='https://www.pisosmadrid.com.es/propiedad/4044649-estudio-en-venta-en-nueva-espana-madrid',"banos"]=1
df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/93990153/',["garaje","trastero"]]=["2 privado",1]
df_final.loc[df_final['url']=='https://www.pisos.com/comprar/piso-casco_historico_de_vallecas28031-39169651748_101800/',["estado","calefaccion","puerta_seguridad","aire_acondicionado","grupo_cocina"]]=["buen estado","individual",1,"1","independiente equipada"]

df_final.loc[df_final['url']=='https://www.redpiso.es/inmueble/piso-en-venta-en-casco-historico-de-vallecas-villa-de-vallecas-ensanche-y-santa-eugenia-madrid-madrid-RP532024131243',["antiguedad","estado","letra_ce","terraza","trastero"]]=[1979,"reformado","E","1",0]
# Cambiar este valor por que no puede ser mayor los m2_utiles que los construidos
# Asignar NaN a 'm2_utiles' cuando 'url' es igual al valor indicado
df_final.loc[df_final['url'] == 'https://www.pisos.com/comprar/piso-centro_palacio28013-48428210263_100500/', 'm2_utiles'] = np.nan
# Cambiar este valor por que no puede ser mayor los m2_utiles que los construidos

df_final.loc[df_final['url'] == 'https://www.pisos.com/comprar/chalet-piovera28043-45869946604_108700/', 'm2_utiles'] = 1150
df_final.loc[df_final['url'] == 'https://www.pisos.com/comprar/piso-salamanca_castellana28006-41697553178_996152/', 'm2_utiles'] = 462
df_final.loc[df_final['url'] == 'https://www.pisos.com/comprar/piso-recoletos28001-44168048071_996152/', 'm2_utiles'] = 323

df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/96571765/',['letra_ce','tipologia','estado','garaje','antiguedad','tiene_armario','exterior','piscina','orientacion_oeste','armarios','cocina','portero','banos']]=['A','Atico','En Buen Estado','4 privado','Entre 5 y 10 años',1,1,'1 propia y 2 comunitaria',1,5,'independiente , amuelada y equipada','todo el dia',6]

df_final.loc[df_final['url']=='https://www.pisos.com/comprar/casa_pareada-moncloa_aravaca_aravaca28023-48337093593_108900/',['garaje','jardin','cancha_tenis','tiene_armario','exterior','estado','piscina','armarios','cocina','portero']]=['2 privado',1,1,1,1,'reformado',1,4,'independiente , amuelada y equipada','todo el dia']

df_final.loc[df_final['url']=='https://www.pisosmadrid.com.es/propiedad/3895830-piso-en-venta-en-lucero-madrid','nuda_propiedad']=1
df_final.loc[df_final['url']=='https://www.indomio.es/anuncios/96758185/','nuda_propiedad']=1

df_final.loc[df_final['url']=='https://www.pisosmadrid.com.es/propiedad/3967137-piso-en-venta-en-lucero-madrid','nuda_propiedad']=1


df_final.loc[df_final['url']=='https://www.redpiso.es/inmueble/piso-en-venta-en-calle-juan-alvarez-de-mendizabal-arguelles-moncloa-aravaca-madrid-madrid-RP71202192163',['dormitorios','banos','m2_constr','calefaccion','estado','planta','ascensor','aire_acondicionado','armarios','antiguedad','letra_ce']]=[2,2,120,'central','reformado',2,1,1,1,1950,'E']

df_final.loc[df_final['url']=='https://www.pisosmadrid.com.es/propiedad/3967137-piso-en-venta-en-lucero-madrid',['m2_constr','estado','piscina','portero','garaje','armarios','antiguedad','terraza','cancha_tenis','trastero']]=[75,'reformado',1,1,1,2,2005,1,1,1]


C:\Users\extas\AppData\Local\Temp\ipykernel_21996\463572899.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[df_final['url']=='https://www.pisos.com/comprar/chalet-fuencarral_el_pardo_el_pardo28023-46668989227_102100/',["piscina","calefaccion","estado","tipologia","exterior","chimenea","jardin","tiene_jardin","cancha_tenis"]]=['propia','suelo radiante','perfecto estado','Chalet',1,1,"privado",'1',1]
C:\Users\extas\AppData\Local\Temp\ipykernel_21996\463572899.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[df_final['url']=='https://www.redpiso.es/inmueble/piso-en-venta-en-casco-historico-de-vallecas-villa

In [4808]:
df_final.query("url=='https://www.indomio.es/anuncios/94254871/'")

,title,url,precio_anterior,descuento,EUR/m2,m2_constr,distrito,cod_distrito,barrio,cod_barrio,zona,calle,consumoce_ano,letra_ce,emisiones_co2,tipologia,estado,amueblado,planta,dormitorios,banos,balcon,terraza,garaje,trastero,ascensor,calefaccion,antiguedad,num_plantas,aire_acondicionado,jardin,inmueble_ingresos,alquiler_opcion_a_compra,nuda_propiedad,tipo_inmueble,tiene_armario,cancha_tenis,carpinteria_exterior_doble_vidrio/pvc,carpinteria_exterior_doble_vidrio/madera,carpinteria_exterior_doble_vidrio/metal,carpinteria_exterior_triple_vidrio/madera,carpinteria_exterior_triple_vidrio/metal,carpinteria_exterior_vidrio/pvc,carpinteria_exterior_vidrio/madera,carpinteria_exterior_vidrio/metal,chimenea,cocina,fibra_optica,exterior,interior,interior_y_exterior,piscina,porton_electrico,alarma,videoportero,acceso_discapacidad,precio,web,fecha_descarga,actualizacion,armarios,codigo_Postal,m2_utiles,referencia,tipo_suelo,certificado_energetico,tiene_jardin,descripcion_larga,num_visitas,fecha_publicacion,titularanuncio,num_pisos,puerta_seguridad,zonas_verdes,gastos_comunidad,tiene_piscina,tipo_fachada,agua_caliente,nombre_oficina,portero,orientacion_este,orientacion_oeste,orientacion_sur,num_garajes,clasificacion_aire,grupo_cocina,orientacion_norte,emisiones_ano,letra_emisiones,tipo_calefaccion,energia_calefaccion,instalacion_calefaccion,titular_anuncio,cod_postal
238,"Piso de cuatro habitaciones Paseo Castellana, Almagro, Madrid",https://www.indomio.es/anuncios/94254871/,NaN,NaN,10.90,624.00,Chamberí,7,Almagro,74,Almagro,"CASTELLANA, ALMAGRO, MADRID",NaN,NaN,NaN,Piso,Bueno / Habitable,NaN,2,4.00,5,NaN,1.00,NaN,1.00,1.00,NaN,1954,NaN,"Individual, frío/calor",sin jardin,NaN,NaN,NaN,lujo,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,NaN,0.00,0.00,0.00,0.00,sin piscina,0.00,0,0.00,<NA>,6800000,Indomio.com,2024-11-20,NaN,NaN,28010.00,NaN,NaN,NaN,No indicado,0.00,"Diplomatic Real Estate ofrece fantástica vivienda de lujo con reforma integral, calidades excepcionales, totalmente aislada de ruido, de diseño, ubicado en edificio clásico y señorial que data de 1954. Se distribuye de la siguiente manera; Amplio recibidor con un espacio acristalado para un jardín / invernadero, aseo de cortesía, y a la izquierda se encuentran tres salones independientes con puertas correderas que se pueden comunicar entre sí, uno de ellos con chimenea. Dispone de puerta de acceso a una terraza rectangular / balcón corrido de 8 m2.El comedor es abierto y dispone de una cocina de apoyo. Un espacio único con distintos ambientes. La zona de noche la compone tres dormitorios en suite, el principal dispone también de un excelente vestidor y bañera exenta además de una importante ducha y dos lavabos. El segundo dormitorio (en suite) tiene un closet además de armarios empotrados. El tercer dormitorio (también en suite) tiene una pared de armarios de gran almacenaje. La zona de servicio está compuesta por una amplia cocina en isla totalmente equipada y amueblada con electrodomésticos de alta gama + puerta de servicio al montacargas + patio acristalado para disfrutar de deliciosos desayunos + zona de lavado + cuarto de plancha + dormitorio de servicio junto con su correspondiente baño. Se vende con una plaza de garaje dentro del propio edificio y trastero. El portero vive en la finca. Esperamos su llamada, 91 564 77 00 Diplomatic Real Estate con más de 25 años en el sector inmobiliario y una amplia cartera de viviendas de lujo, locales, oficinas y edificios ofrece asesoramiento completo a sus clientes tanto en alquiler como en compra venta de inmuebles.",NaN,NaN,NaN,NaN,0,NaN,NaN,0.00,NaN,NaN,NaN,NaN,0.00,0.00,0.00,NaN,Normal,sin_informacion,0.00,NaN,NaN,No clasif,No clasificado en energía,Radiadores,"Diplomatic Real Estate ofrece fantástica vivienda de lujo con reforma integral, calidades excepcionales, totalmente aislada de ruido, de diseño, ubicado en edificio clásico y señorial que data de 1954. Se distribuye de la siguiente manera; Amplio recibidor con un espaci

In [4809]:
# Asignar valores según las siguientes condiciones:
df_final['certificado_energetico'] = df_final['certificado_energetico'].replace({
    'Disponible': 1,
    'No indicado': 0
})
df_final['certificado_energetico'] = df_final['certificado_energetico'].fillna(0)

# Asegurarse de que todos los valores sean tratados de forma uniforme
df_final['certificado_energetico'] = df_final['certificado_energetico'].astype(float)

# También se puede reemplazar 0.0 por 0 (si quieres mantenerlo como entero)
df_final['certificado_energetico'] = df_final['certificado_energetico'].replace(0.0, 0)

# Verificar los cambios
print(df_final['certificado_energetico'].value_counts())

certificado_energetico
0.00    8144
1.00    1888
Name: count, dtype: int64


In [4810]:
df_final["distrito"]=df_final["distrito"].str.strip()
df_final["barrio"]=df_final["barrio"].str.strip()

In [4811]:
df_final["distrito"].value_counts(dropna=False)

distrito
Centro                 1755
Salamanca              1514
Chamartín               636
Chamberí                605
Carabanchel             537
Puente De Vallecas      525
Tetuán                  464
Fuencarral-El Pardo     424
Retiro                  410
Latina                  406
Ciudad Lineal           397
Moncloa-Aravaca         376
San Blas-Canillejas     351
Arganzuela              294
Villaverde              291
Hortaleza               290
Villa De Vallecas       232
Usera                   190
Vicálvaro               132
Moratalaz               103
Barajas                 100
Name: count, dtype: int64

In [4812]:
# Supongamos que tienes un DataFrame llamado df
# Establecer el valor en 'tipo_suelo' como 'suelo radiante' si 'instalacion calefaccion' es 'suelo radiante'
df_final.loc[df_final['instalacion_calefaccion'] == 'Suelo Radiante', 'tipo_suelo'] = 'Suelo radiante'
df_final.loc[df_final['tipo_suelo'] == 'Suelo radiante', 'tipo_suelo'] = 'Suelo Radiante'

In [4813]:
df_final=df_final.drop(columns='num_garajes')
df_final=df_final.drop(columns='interior_y_exterior')

In [4814]:
cols=['amueblado','puerta_seguridad','piscina','jardin','garaje']
for c in cols:
  df_final[c]=df_final[c].astype(str)

In [4815]:

df_final.reset_index(drop=True, inplace=True)


dfEDA = df_final
# Ruta relativa del archivo
ruta_guardado_relativa = '../../02-EDA-y-feature-engineering/data/pisosEDA.csv'

# Crear el directorio de destino si no existe
directorio_destino = os.path.dirname(ruta_guardado_relativa)
os.makedirs(directorio_destino, exist_ok=True)

# Guardar el DataFrame dfEDA en la ruta relativa especificada
dfEDA.to_csv(ruta_guardado_relativa,sep=";", index=False)